In [ ]:
!pip install -U fsspec gcsfs
!pip install -U \
  "transformers==4.57.3" \
  "accelerate>=1.0.0" \
  "peft==0.9.0" \
  "datasets==2.17.0" \
  "bitsandbytes==0.48.2"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.0
    Uninstalling gcsfs-2025.3.0:
      Successfully uninstalled gcsfs-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.2 requires fsspec[http]<=2025.10.0,>=2023.1.0, but you have fsspec 2026.1.0 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2026.1.0
    Uninstalling fsspec-2026.1.0:
      Successfully uninstalled fsspec-2026.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.2
    Uninstalling datasets-4.4.2:
      Successfully uninstalled datasets-4.4.2
  Attempting uninstall: peft
    Found existing installation: peft 0.18.0
    Uninstalling peft-0.18.0:
      Successfully uninstalled peft-0.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2026.1.0 requires fsspec==202

In [ ]:
# Flash Attention 2 yüklemesi (A100 için kritiktir)
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 108.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [1]:
# @title
# ================== IMPORTLAR ==================
import os
import time
import json
from datetime import timedelta
import sys

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    default_data_collator,
    pipeline,
)

import peft
import PIL

# bitsandbytes varsa sürümünü de yazalım
try:
    import bitsandbytes as bnb
except ImportError:
    bnb = None

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

if bnb is not None:
    safe_version("bitsandbytes", bnb)
else:
    print("bitsandbytes: yüklü değil")
print(PIL.__version__)
print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

==== Python ve Temel Bilgiler ====
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

==== Ana Kütüphaneler ====
pandas version: 2.2.2
torch version: 2.9.0+cu126
PIL version: 11.3.0
datasets version: 4.0.0
transformers version: 5.0.0
peft version: 0.18.1
bitsandbytes: yüklü değil
11.3.0

==== GPU Durumu ====
torch.cuda.is_available(): True
CUDA device count: 1
Current device: 0
Device name: NVIDIA A100-SXM4-40GB


In [ ]:
!umount -f /content/drive
!rm -rf /content/drive

umount: /content/drive: not mounted.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# @title
# =========================
# QWEN2.5-VL COLAB CELL (OOM-SAFE + FIXED, 4-bit, TWO-STAGE)
# - Fix: NO truncation in processor to avoid image token mismatch
# - OOM handled via checkpointing + cache off + safer args
# =========================

import os
import time
import json
from datetime import timedelta
import sys
from functools import lru_cache

import pandas as pd
import torch
from PIL import Image as PImage

from datasets import Dataset
import transformers
from transformers import (
    AutoProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    set_seed,
)

from transformers import Qwen2_5_VLForConditionalGeneration

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


# ================== SPEED (SAFE) ==================
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("transformers", transformers)
safe_version("peft", peft)

# ================== 4-BIT ZORUNLU ==================
try:
    import bitsandbytes as bnb
    safe_version("bitsandbytes", bnb)
except ImportError:
    print("❌ bitsandbytes yüklü değil. 4-bit zorunlu.")
    print("✅ Çözüm: pip install -U bitsandbytes")
    raise

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU yok. Colab: Runtime > Change runtime type > GPU")
    sys.exit(0)
print("GPU:", torch.cuda.get_device_name(0))

# ===================== 0) SEED =====================
SEED = int(os.environ.get("SEED", "42"))
set_seed(SEED)

# ===================== 1) PATH + PARAMETRELER (COLAB) =====================
DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TRAIN_QA_FILE = os.path.join(DATA_ROOT, "All_QA_Pairs_train.txt")
VAL_QA_FILE   = os.path.join(DATA_ROOT, "All_QA_Pairs_val.txt")

TRAIN_IMAGE_DIR = os.path.join(DATA_ROOT, "Train_images")
VAL_IMAGE_DIR   = os.path.join(DATA_ROOT, "Val_images")

IMAGE_EXTENSION = os.environ.get("IMAGE_EXTENSION", ".jpg")

_stamp = time.strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = os.environ.get(
    "OUTPUT_DIR",
    os.path.join("/content/drive/MyDrive", f"qwen_vl_train_output_oomsafe_{_stamp}")
)
os.makedirs(OUTPUT_DIR, exist_ok=True)

MODEL_ID = os.environ.get("MODEL_ID", "Qwen/Qwen2.5-VL-7B-Instruct")
SYSTEM_PROMPT = os.environ.get("SYSTEM_PROMPT", "You are a medical VQA assistant.")

# Two-stage epochs (OOM SAFE defaults)
STAGE1_EPOCHS = int(os.environ.get("STAGE1_EPOCHS", "4"))
STAGE2_EPOCHS = int(os.environ.get("STAGE2_EPOCHS", "1"))  # OOM ise 0 yap

ATTN_IMPL = os.environ.get("ATTN_IMPL", "")

print("\n==== PATH KONTROL ====")
print("DATA_ROOT       :", DATA_ROOT)
print("TRAIN_QA_FILE   :", TRAIN_QA_FILE)
print("VAL_QA_FILE     :", VAL_QA_FILE)
print("TRAIN_IMAGE_DIR :", TRAIN_IMAGE_DIR)
print("VAL_IMAGE_DIR   :", VAL_IMAGE_DIR)
print("OUTPUT_DIR      :", OUTPUT_DIR)
print("MODEL_ID        :", MODEL_ID)
print("STAGE1_EPOCHS   :", STAGE1_EPOCHS)
print("STAGE2_EPOCHS   :", STAGE2_EPOCHS)
print("ATTN_IMPL       :", ATTN_IMPL if ATTN_IMPL else "(default)")
print("SEED            :", SEED)

for p in [DATA_ROOT, TRAIN_QA_FILE, VAL_QA_FILE, TRAIN_IMAGE_DIR, VAL_IMAGE_DIR]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"Bulunamadı: {p}")

# ===================== 2) DATA OKUMA =====================
def read_pipe_qa(path_txt: str) -> pd.DataFrame:
    last_err = None
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            rows = []
            with open(path_txt, "r", encoding=enc, errors="replace") as f:
                for line in f:
                    line = line.rstrip("\n\r")
                    if not line.strip():
                        continue
                    parts = line.split("|", 2)
                    if len(parts) < 3:
                        continue
                    image_id = parts[0].strip()
                    question = parts[1].strip()
                    answer   = parts[2].strip()
                    rows.append((image_id, question, answer))
            return pd.DataFrame(rows, columns=["image_id", "question", "answer"])
        except Exception as e:
            last_err = e
    raise RuntimeError(f"QA dosyası okunamadı: {path_txt} (son hata: {last_err})")

def load_qa_file(path_txt: str, image_dir: str, name: str) -> pd.DataFrame:
    print(f"\n🔹 {name} TXT okunuyor:", path_txt)

    df = read_pipe_qa(path_txt)
    df["image_id"] = df["image_id"].astype(str).str.strip()
    df["question"] = df["question"].astype(str).str.strip()
    df["answer"]   = df["answer"].astype(str).str.strip()

    df["image_path"] = df["image_id"].apply(lambda x: os.path.join(image_dir, f"{x}{IMAGE_EXTENSION}"))
    df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

    if len(df) == 0:
        raise RuntimeError(f"{name} için hiç geçerli örnek yok. Klasör/uzantı yanlış olabilir.")

    print(f"{name} QA sayısı     : {len(df)}")
    print(f"{name} unique images : {df['image_id'].nunique()}")
    return df

df_train = load_qa_file(TRAIN_QA_FILE, TRAIN_IMAGE_DIR, "TRAIN")
df_val   = load_qa_file(VAL_QA_FILE,   VAL_IMAGE_DIR,   "VAL")

# ===================== 3) MODEL + PROCESSOR =====================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_kwargs = dict(
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
)
if ATTN_IMPL.strip():
    model_kwargs["attn_implementation"] = ATTN_IMPL.strip()

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(MODEL_ID, **model_kwargs)

processor = AutoProcessor.from_pretrained(MODEL_ID)
tokenizer = processor.tokenizer

# 4-bit training prep
model = prepare_model_for_kbit_training(model)

# ===================== OOM-SAFE: CHECKPOINTING + CACHE OFF =====================
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()

# 🔥 çok kritik
model.config.use_cache = False

torch.cuda.empty_cache()

# ===================== 3.1) LoRA target_modules (AUTO SAFE) =====================
def pick_lora_targets(m):
    candidates = [
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj",
        "mm_projector","vision_proj","multi_modal_projector","visual_proj",
    ]
    present = set()
    for name, module in m.named_modules():
        leaf = name.split(".")[-1]
        if leaf in candidates:
            present.add(leaf)
    base = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
    out = []
    for t in base:
        if t not in out:
            out.append(t)
    for t in candidates:
        if t in present and t not in out:
            out.append(t)
    return out

# OOM-SAFE default
LORA_R = int(os.environ.get("LORA_R", "32"))
LORA_ALPHA = int(os.environ.get("LORA_ALPHA", "64"))
LORA_DROPOUT = float(os.environ.get("LORA_DROPOUT", "0.05"))

target_modules = pick_lora_targets(model)
print("\n✅ LoRA target_modules:", target_modules)

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=target_modules,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4) PROMPT (aynı) =====================
def build_messages(question: str):
    return [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": f"Question: {question}\nAnswer briefly with a short phrase."},
        ]},
    ]

PROMPT_ANSWER_SEP = " "

# ===================== Dataset =====================
def format_and_load_dataset(df: pd.DataFrame) -> Dataset:
    prompts, prompt_lens, answers, questions = [], [], [], []
    for _, row in df.iterrows():
        msgs = build_messages(row["question"])

        prompt_text = processor.apply_chat_template(
            msgs,
            tokenize=False,
            add_generation_prompt=True,
        )

        pl = len(tokenizer(prompt_text + PROMPT_ANSWER_SEP, add_special_tokens=True)["input_ids"])

        prompts.append(prompt_text)
        prompt_lens.append(pl)
        answers.append(row["answer"])
        questions.append(row["question"])

    return Dataset.from_dict(
        {
            "image_path": df["image_path"].tolist(),
            "image_id": df["image_id"].tolist(),
            "prompt": prompts,
            "prompt_len": prompt_lens,
            "answer": answers,
            "question": questions,
        }
    )

train_dataset = format_and_load_dataset(df_train)
val_dataset   = format_and_load_dataset(df_val)

print(f"\nEğitim Seti Boyutu : {len(train_dataset)}")
print(f"Validation Seti    : {len(val_dataset)}")

# ===================== 5) IMAGE CACHE =====================
IMG_CACHE_SIZE = int(os.environ.get("IMG_CACHE_SIZE", "1024"))

@lru_cache(maxsize=IMG_CACHE_SIZE)
def _load_rgb_cached(path: str):
    img = PImage.open(path).convert("RGB")
    return img.copy()

# ===================== 6) DATA COLLATOR =====================
# 🔥 Qwen2.5-VL için truncation='max_length' image token mismatch yapabildiği için KAPALI.
# OOM kontrolü için: cevap zaten kısa -> answer'a hard cap koyuyoruz.

MAX_ANS_CHARS = int(os.environ.get("MAX_ANS_CHARS", "96"))  # OOM için küçük tut

def vqa_data_collator(examples):
    images  = [_load_rgb_cached(e["image_path"]) for e in examples]
    prompts = [e["prompt"] for e in examples]
    prompt_lens = [int(e["prompt_len"]) for e in examples]

    eos = tokenizer.eos_token or ""
    # answer kısa kalsın (OOM'a etki eden token sayısını azaltır)
    answers = []
    for e in examples:
        a = e["answer"].strip()
        if len(a) > MAX_ANS_CHARS:
            a = a[:MAX_ANS_CHARS].rstrip()
        a = (a + (eos if eos else "")).strip()
        answers.append(a)

    full_texts = [p + PROMPT_ANSWER_SEP + a for p, a in zip(prompts, answers)]

    proc_out = processor(
        text=full_texts,
        images=images,
        return_tensors="pt",
        padding=True,
        # truncation=False  (default) -> mismatch fix
    )

    batch = dict(proc_out)
    input_ids = batch["input_ids"]
    labels = input_ids.clone()

    for i, pl in enumerate(prompt_lens):
        pl = min(pl, labels.shape[1])
        labels[i, :pl] = -100

    if tokenizer.pad_token_id is not None:
        labels[labels == tokenizer.pad_token_id] = -100

    batch["labels"] = labels
    return batch

# ===================== 7) COMMON TRAIN PARAMS =====================
EVAL_STEPS = int(os.environ.get("EVAL_STEPS", "1000"))
SAVE_STEPS = int(os.environ.get("SAVE_STEPS", "2000"))

DL_WORKERS = int(os.environ.get("DL_WORKERS", "0"))
BATCH_SIZE = int(os.environ.get("BATCH_SIZE", "1"))
GRAD_ACCUM = int(os.environ.get("GRAD_ACCUM", "8"))

PATIENCE = int(os.environ.get("PATIENCE", "4"))
MIN_DELTA = float(os.environ.get("MIN_DELTA", "0.0003"))

SAVE_TOTAL_LIMIT = int(os.environ.get("SAVE_TOTAL_LIMIT", "2"))
MAX_GRAD_NORM = float(os.environ.get("MAX_GRAD_NORM", "0.8"))
WEIGHT_DECAY = float(os.environ.get("WEIGHT_DECAY", "0.001"))

def set_vision_trainable(m, trainable: bool):
    if hasattr(m, "visual"):
        for p in m.visual.parameters():
            p.requires_grad = trainable
        print("✅ model.visual trainable =", trainable)
    else:
        print("⚠ model.visual bulunamadı; atlandı.")

# -------- STAGE-1: vision frozen --------
set_vision_trainable(model, trainable=False)

LR_STAGE1 = float(os.environ.get("LR_STAGE1", "1.5e-5"))
WARMUP_RATIO_STAGE1 = float(os.environ.get("WARMUP_RATIO_STAGE1", "0.03"))
SCHED_STAGE1 = "cosine"

training_args_1 = TrainingArguments(
    output_dir=os.path.join(OUTPUT_DIR, "stage1"),
    num_train_epochs=STAGE1_EPOCHS,

    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,

    per_device_eval_batch_size=1,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,

    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    save_safetensors=True,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    learning_rate=LR_STAGE1,
    warmup_ratio=WARMUP_RATIO_STAGE1,
    lr_scheduler_type=SCHED_STAGE1,

    weight_decay=WEIGHT_DECAY,
    max_grad_norm=MAX_GRAD_NORM,

    optim="adamw_torch",
    fp16=True,

    logging_steps=50,
    report_to="none",
    remove_unused_columns=False,

    dataloader_num_workers=DL_WORKERS,
    dataloader_pin_memory=False,
)

trainer1 = Trainer(
    model=model,
    args=training_args_1,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=vqa_data_collator,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=PATIENCE,
            early_stopping_threshold=MIN_DELTA
        )
    ],
)

print(f"\n🧪 STAGE-1 başlıyor (vision frozen, OOM-safe, mismatch-fix)")
print(f"OUTPUT_DIR={OUTPUT_DIR}")
print(f"epochs={STAGE1_EPOCHS}, LR={LR_STAGE1}, sched={SCHED_STAGE1}, warmup={WARMUP_RATIO_STAGE1}")
print(f"batch={BATCH_SIZE}, grad_accum={GRAD_ACCUM} (effective={BATCH_SIZE*GRAD_ACCUM})")
print(f"eval_steps={EVAL_STEPS}, save_steps={SAVE_STEPS}, patience={PATIENCE}, min_delta={MIN_DELTA}")
print(f"IMG_CACHE_SIZE={IMG_CACHE_SIZE}, workers={DL_WORKERS}, pin_memory=False")
print("use_cache:", model.config.use_cache, "| gradient_checkpointing: enabled (if supported)")
print("MAX_ANS_CHARS:", MAX_ANS_CHARS)

t0 = time.time()
res1 = trainer1.train()
t1 = time.time()
stage1_time = t1 - t0
torch.cuda.empty_cache()

# ===================== 7.2) STAGE-2 (VISION UNFREEZE + LOW LR) =====================
stage2_time = 0.0
res2 = None

if STAGE2_EPOCHS > 0:
    set_vision_trainable(model, trainable=True)

    LR_STAGE2 = float(os.environ.get("LR_STAGE2", "6e-6"))
    WARMUP_RATIO_STAGE2 = float(os.environ.get("WARMUP_RATIO_STAGE2", "0.02"))
    SCHED_STAGE2 = "cosine"

    training_args_2 = TrainingArguments(
        output_dir=os.path.join(OUTPUT_DIR, "stage2"),
        num_train_epochs=STAGE2_EPOCHS,

        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,

        per_device_eval_batch_size=1,
        eval_strategy="steps",
        eval_steps=EVAL_STEPS,

        save_strategy="steps",
        save_steps=SAVE_STEPS,
        save_total_limit=SAVE_TOTAL_LIMIT,
        save_safetensors=True,

        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,

        learning_rate=LR_STAGE2,
        warmup_ratio=WARMUP_RATIO_STAGE2,
        lr_scheduler_type=SCHED_STAGE2,

        weight_decay=WEIGHT_DECAY,
        max_grad_norm=MAX_GRAD_NORM,

        optim="adamw_torch",
        fp16=True,

        logging_steps=50,
        report_to="none",
        remove_unused_columns=False,

        dataloader_num_workers=DL_WORKERS,
        dataloader_pin_memory=False,
    )

    trainer2 = Trainer(
        model=model,
        args=training_args_2,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=vqa_data_collator,
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_patience=max(2, PATIENCE // 2),
                early_stopping_threshold=MIN_DELTA
            )
        ],
    )

    print(f"\n🧪 STAGE-2 başlıyor (vision unfrozen, OOM-safe)")
    print(f"epochs={STAGE2_EPOCHS}, LR={LR_STAGE2}, sched={SCHED_STAGE2}, warmup={WARMUP_RATIO_STAGE2}")

    t2 = time.time()
    res2 = trainer2.train()
    t3 = time.time()
    stage2_time = t3 - t2
    torch.cuda.empty_cache()

    final_trainer = trainer2
else:
    print("\n🟡 STAGE-2 atlandı (STAGE2_EPOCHS=0). OOM için en güvenlisi.")
    final_trainer = trainer1

# ===================== 8) FINAL EVAL + LOG =====================
eval_metrics = final_trainer.evaluate(val_dataset)

total_time_sec = (stage1_time + stage2_time)
metrics = {
    "seed": SEED,
    "stage1_time_sec": float(stage1_time),
    "stage2_time_sec": float(stage2_time),
    "total_training_time_sec": float(total_time_sec),
    "total_training_time_str": str(timedelta(seconds=int(total_time_sec))),
    "num_train_samples": len(train_dataset),
    "num_eval_samples": len(val_dataset),
    "stage1_global_step": int(getattr(res1, "global_step", trainer1.state.global_step)),
    "stage2_global_step": int(getattr(res2, "global_step", final_trainer.state.global_step)) if res2 is not None else 0,
    "use_cache": bool(getattr(model.config, "use_cache", False)),
    "BATCH_SIZE": BATCH_SIZE,
    "GRAD_ACCUM": GRAD_ACCUM,
    "DL_WORKERS": DL_WORKERS,
    "IMG_CACHE_SIZE": IMG_CACHE_SIZE,
    "MAX_ANS_CHARS": MAX_ANS_CHARS,
}
for k, v in eval_metrics.items():
    metrics[f"eval_{k}"] = float(v) if isinstance(v, (int, float)) else v

log_obj = {
    "final_metrics": metrics,
    "stage1_training_args": training_args_1.to_dict(),
    "log_history": final_trainer.state.log_history,
}
if STAGE2_EPOCHS > 0:
    log_obj["stage2_training_args"] = training_args_2.to_dict()

log_path = os.path.join(OUTPUT_DIR, "training_log_oomsafe_fixed.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_qwen_oomsafe_fixed")
final_trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Eğitim bitti.")
print(f"✅ Adaptör: {ADAPTER_PATH}")
print(f"✅ Checkpointler: {OUTPUT_DIR}/checkpoint-*")


In [ ]:
# @title
# =========================
# QWEN2.5-VL TEST (JSON OUT) - LOAD SPECIFIC CHECKPOINT
# checkpoint: qwen_vl_train_output_oomsafe_20260106_131859/stage1/checkpoint-6396
# IMPORTANT: no truncation to avoid image token mismatch
# =========================

import os, json, time
from typing import Dict, Any, List
import pandas as pd
import torch
from PIL import Image as PImage

from transformers import AutoProcessor, BitsAndBytesConfig
from transformers import Qwen2_5_VLForConditionalGeneration
from peft import PeftModel

# -------------------------
# 0) PATHS (fixed for your checkpoint)
# -------------------------
DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TEST_QA_FILE    = os.path.join(DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR  = os.path.join(DATA_ROOT, "VQAMed2019_Test_Images")
IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR = "/content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859"

# ✅ FORCE checkpoint adapter
CKPT_PATH = "/content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859/stage1/checkpoint-6396"

# Base model id MUST match training
MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct"

# Prompt style must match training
SYSTEM_PROMPT = "You are a medical VQA assistant."
PROMPT_ANSWER_SEP = " "

# Generation params
MAX_NEW_TOKENS = 32
DO_SAMPLE = False  # deterministic
TEMPERATURE = 0.2
TOP_P = 0.9

# Output files
_stamp = time.strftime("%Y%m%d_%H%M%S")
OUT_FLAT = os.path.join(OUTPUT_DIR, f"qwen_test_preds_flat_ckpt6396_{_stamp}.json")
OUT_GROUPED = os.path.join(OUTPUT_DIR, f"qwen_test_preds_grouped_ckpt6396_{_stamp}.json")

print("TEST_QA_FILE   :", TEST_QA_FILE)
print("TEST_IMAGE_DIR :", TEST_IMAGE_DIR)
print("OUTPUT_DIR     :", OUTPUT_DIR)
print("CKPT_PATH      :", CKPT_PATH)
print("MODEL_ID       :", MODEL_ID)
print("OUT_FLAT       :", OUT_FLAT)
print("OUT_GROUPED    :", OUT_GROUPED)

for p in [TEST_QA_FILE, TEST_IMAGE_DIR, OUTPUT_DIR, CKPT_PATH]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"Bulunamadı: {p}")

# -------------------------
# 1) Read QA file (pipe-delimited: image|question|answer)
# -------------------------
def read_pipe_qa(path_txt: str) -> pd.DataFrame:
    last_err = None
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            rows = []
            with open(path_txt, "r", encoding=enc, errors="replace") as f:
                for line in f:
                    line = line.rstrip("\n\r")
                    if not line.strip():
                        continue
                    parts = line.split("|", 2)
                    if len(parts) < 3:
                        continue
                    image_id = parts[0].strip()
                    question = parts[1].strip()
                    answer   = parts[2].strip()
                    rows.append((image_id, question, answer))
            return pd.DataFrame(rows, columns=["image_id", "question", "answer"])
        except Exception as e:
            last_err = e
    raise RuntimeError(f"QA dosyası okunamadı: {path_txt} (son hata: {last_err})")

df_test = read_pipe_qa(TEST_QA_FILE)
df_test["image_id"] = df_test["image_id"].astype(str).str.strip()
df_test["question"] = df_test["question"].astype(str).str.strip()
df_test["answer"]   = df_test["answer"].astype(str).str.strip()
df_test["image_path"] = df_test["image_id"].apply(lambda x: os.path.join(TEST_IMAGE_DIR, f"{x}{IMAGE_EXTENSION}"))
df_test = df_test[df_test["image_path"].apply(os.path.exists)].reset_index(drop=True)

print("TEST rows:", len(df_test), "| unique images:", df_test["image_id"].nunique())
if len(df_test) == 0:
    raise RuntimeError("TEST set 0 örnek. TEST_IMAGE_DIR / IMAGE_EXTENSION yanlış olabilir.")

# -------------------------
# 2) Load model (4-bit) + processor + LoRA adapter from checkpoint
# -------------------------
if not torch.cuda.is_available():
    raise RuntimeError("GPU yok. Colab: Runtime > Change runtime type > GPU")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

base = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
)

processor = AutoProcessor.from_pretrained(MODEL_ID)
tokenizer = processor.tokenizer

model = PeftModel.from_pretrained(base, CKPT_PATH)
model.eval()
model.config.use_cache = True  # inference

device = next(model.parameters()).device
print("✅ Loaded adapter checkpoint:", CKPT_PATH)
print("✅ Device:", device)

# -------------------------
# 3) Prompt builder (must match training)
# -------------------------
def build_messages(question: str):
    return [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": f"Question: {question}\nAnswer briefly with a short phrase."},
        ]},
    ]

def decode_answer(gen_ids: torch.Tensor, prompt_len: int) -> str:
    out = gen_ids[0][prompt_len:]
    text = tokenizer.decode(out, skip_special_tokens=True).strip()
    return " ".join(text.split())

# -------------------------
# 4) Inference loop (OOM-safe: 1 sample at a time)
# -------------------------
preds_flat: Dict[str, List[Dict[str, Any]]] = {}

with torch.no_grad():
    for i, row in df_test.iterrows():
        image_id = row["image_id"]
        q  = row["question"]
        gt = row["answer"]
        img_path = row["image_path"]

        img = PImage.open(img_path).convert("RGB")

        msgs = build_messages(q)
        prompt_text = processor.apply_chat_template(
            msgs,
            tokenize=False,
            add_generation_prompt=True,
        )

        # IMPORTANT: no truncation
        inputs = processor(
            text=prompt_text,
            images=img,
            return_tensors="pt",
            padding=True,
        )

        # move tensors to model device
        for k, v in list(inputs.items()):
            if torch.is_tensor(v):
                inputs[k] = v.to(device)

        prompt_len = int(inputs["input_ids"].shape[1])

        gen = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=DO_SAMPLE,
            temperature=TEMPERATURE if DO_SAMPLE else None,
            top_p=TOP_P if DO_SAMPLE else None,
            num_beams=1,
        )

        pred = decode_answer(gen, prompt_len)

        preds_flat.setdefault(image_id, []).append({
            "question": q,
            "ground_truth": gt,
            "predicted": pred,
        })

        if (i + 1) % 200 == 0:
            print(f"[{i+1}/{len(df_test)}] last_pred:", pred)
            torch.cuda.empty_cache()

# -------------------------
# 5) Save JSONs
# -------------------------
with open(OUT_FLAT, "w", encoding="utf-8") as f:
    json.dump(preds_flat, f, ensure_ascii=False, indent=2)
print("✅ Saved flat:", OUT_FLAT)

# grouped = same (image_id grouped)
with open(OUT_GROUPED, "w", encoding="utf-8") as f:
    json.dump(preds_flat, f, ensure_ascii=False, indent=2)
print("✅ Saved grouped:", OUT_GROUPED)

print("\nDONE.")


`torch_dtype` is deprecated! Use `dtype` instead!


TEST_QA_FILE   : /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/VQAMed2019_Test_Questions_w_Ref_Answers.txt
TEST_IMAGE_DIR : /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/VQAMed2019_Test_Images
OUTPUT_DIR     : /content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859
CKPT_PATH      : /content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859/stage1/checkpoint-6396
MODEL_ID       : Qwen/Qwen2.5-VL-7B-Instruct
OUT_FLAT       : /content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859/qwen_test_preds_flat_ckpt6396_20260107_112316.json
OUT_GROUPED    : /content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859/qwen_test_preds_grouped_ckpt6396_20260107_112316.json
TEST rows: 500 | unique images: 500


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

✅ Loaded adapter checkpoint: /content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859/stage1/checkpoint-6396
✅ Device: cuda:0
[200/500] last_pred: ap
[400/500] last_pred: pulmonary embolism
✅ Saved flat: /content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859/qwen_test_preds_flat_ckpt6396_20260107_112316.json
✅ Saved grouped: /content/drive/MyDrive/qwen_vl_train_output_oomsafe_20260106_131859/qwen_test_preds_grouped_ckpt6396_20260107_112316.json

DONE.


In [ ]:
# @title
import os, json, pandas as pd, torch, gc, threading, time, psutil
from PIL import Image as PImage
from datasets import Dataset
from datetime import timedelta
from transformers.trainer_utils import get_last_checkpoint
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    EarlyStoppingCallback
)

# --- 1. SİSTEM TEMİZLİĞİ ---
gc.collect()
torch.cuda.empty_cache()

# --- 2. CALLBACK'LER ---
class PrecisionEarlyStopping(EarlyStoppingCallback):
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if state.global_step > 0 and state.global_step % 200 == 0:
            super().on_evaluate(args, state, control, metrics, **kwargs)
            if control.should_training_stop:
                current_epoch = state.epoch
                if current_epoch % 1.0 > 0.02:
                    print(f"\n[BİLGİ]: Durdurma ertelendi. Epoch {int(current_epoch)+1} tamamlanacak.")
                    control.should_training_stop = False
        else:
            control.should_training_stop = False

class JsonLoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
        self.history = []
        if os.path.exists(log_path):
            try:
                with open(log_path, "r") as f: self.history = json.load(f)
                print(f"✅ Eski loglar yüklendi: {len(self.history)} kayıt bulundu.")
            except: pass

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            report = {"step": state.global_step, "epoch": state.epoch,
                      "train_loss": logs.get("loss"), "eval_loss": logs.get("eval_loss"),
                      "learning_rate": logs.get("learning_rate")}
            if any(v is not None for v in [report["train_loss"], report["eval_loss"]]):
                self.history.append(report)
                try:
                    with open(self.log_path, "w") as f: json.dump(self.history, f, indent=4)
                except: pass

# --- 3. KALP ATIŞI MONİTÖRÜ ---
def stay_alive_monitor(stop_event):
    print("\n💓 Heartbeat Monitor Başlatıldı...")
    start_time = time.time()
    while not stop_event.is_set():
        current_ram = psutil.virtual_memory().used / (1024**3)
        elapsed = (time.time() - start_time) / 60
        print(f"⏳ {elapsed:.1f} dk | RAM: {current_ram:.2f} GB / 167 GB | Durum: Yükleniyor...")
        time.sleep(30)

# --- 4. DRIVE VE VERİ ---
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

OUT_DIR = "/content/drive/MyDrive/llava_fullft_zero2_out"
DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"
LOG_JSON_FILE = os.path.join(OUT_DIR, "training_metrics.json")
MODEL_ID = "llava-hf/llava-1.5-7b-hf"
os.makedirs(OUT_DIR, exist_ok=True)

processor = LlavaProcessor.from_pretrained(MODEL_ID)
tokenizer = processor.tokenizer

def read_pipe_qa(path_txt):
    rows = []
    with open(path_txt, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split("|", 2)
            if len(parts) == 3: rows.append((parts[0].strip(), parts[1].strip(), parts[2].strip()))
    return pd.DataFrame(rows, columns=["image_id", "question", "answer"])

df_train = read_pipe_qa(os.path.join(DATA_ROOT, "All_QA_Pairs_train.txt"))
df_val = read_pipe_qa(os.path.join(DATA_ROOT, "All_QA_Pairs_val.txt"))

def build_dataset(df):
    prompts = [f"USER: <image>\nQuestion: {q}\nAnswer briefly.\nASSISTANT:" for q in df["question"]]
    def get_path(x):
        p1 = os.path.join(DATA_ROOT, "Train_images", f"{x}.jpg")
        p2 = os.path.join(DATA_ROOT, "Val_images", f"{x}.jpg")
        return p1 if os.path.exists(p1) else p2
    return Dataset.from_dict({
        "image_path": df["image_id"].apply(get_path).tolist(),
        "prompt": prompts,
        "answer": df["answer"].tolist(),
        "prompt_len": [len(tokenizer(p, add_special_tokens=False)["input_ids"]) for p in prompts]
    })

train_ds = build_dataset(df_train); val_ds = build_dataset(df_val)

# --- 5. MODEL ---
model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, attn_implementation="flash_attention_2"
)
model.config.use_cache = False
model.gradient_checkpointing_enable()

def collate_fn(examples):
    images = []; valid_items = []
    for e in examples:
        try:
            img = PImage.open(e["image_path"]).convert("RGB")
            images.append(img); valid_items.append(e)
        except: continue
    if not images: return None
    full_texts = [f"{e['prompt']} {e['answer']}{tokenizer.eos_token}" for e in valid_items]
    batch = processor(text=full_texts, images=images, padding=True, return_tensors="pt")
    labels = batch["input_ids"].clone()
    for i, pl in enumerate([e["prompt_len"] for e in valid_items]): labels[i, :pl] = -100
    labels[labels == tokenizer.pad_token_id] = -100
    batch["labels"] = labels
    return batch

# --- 6. TRAINING ARGUMENTS (RAM OPTIMIZED) ---
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    bf16=True,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=0,
    deepspeed={
        "zero_optimization": {
            "stage": 2,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": False # RAM'i %15 rahatlatır
            },
            "allgather_partitions": True,
            "allgather_bucket_size": 2e8, # Küçültüldü
            "overlap_comm": False,        # RAM spike engeller
            "reduce_scatter": True,
            "contiguous_gradients": True,
            "round_robin_gradients": True # Bellek yükünü dağıtır
        },
        "bf16": {"enabled": True},
        "train_batch_size": "auto"
    },
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_ds, eval_dataset=val_ds,
    data_collator=collate_fn,
    callbacks=[PrecisionEarlyStopping(early_stopping_patience=4), JsonLoggingCallback(LOG_JSON_FILE)]
)

# --- 7. BAŞLAT ---
last_checkpoint = get_last_checkpoint(OUT_DIR)
stop_event = threading.Event()
monitor_thread = threading.Thread(target=stay_alive_monitor, args=(stop_event,))

try:
    if last_checkpoint:
        print(f"📦 Checkpoint Resuming: {last_checkpoint}")
        monitor_thread.start()
        trainer.train(resume_from_checkpoint=last_checkpoint)
    else:
        trainer.train()
finally:
    stop_event.set()
    if monitor_thread.is_alive(): monitor_thread.join()

trainer.save_model(OUT_DIR)
processor.save_pretrained(OUT_DIR)
print(f"✅ Tamamlandı.")

Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Eski loglar yüklendi: 60 kayıt bulundu.
📦 Checkpoint Resuming: /content/drive/MyDrive/llava_fullft_zero2_out/checkpoint-1400

💓 Heartbeat Monitor Başlatıldı...
⏳ 0.0 dk | RAM: 17.59 GB / 167 GB | Durum: Yükleniyor...
Installed CUDA version 12.5 does not match the version torch was compiled with 12.6 but since the APIs are compatible, accepting this combination
⏳ 0.5 dk | RAM: 12.73 GB / 167 GB | Durum: Yükleniyor...
Before initializing optimizer states
MA 13.4 GB         Max_MA 13.4 GB         CA 13.42 GB         Max_CA 13 GB 
CPU Virtual Memory:  used = 46.12 GB, percent = 27.6%
After initializing optimizer states
MA 13.4 GB         Max_MA 13.4 GB         CA 13.42 GB         Max_CA 13 GB 
CPU Virtual Memory:  used = 72.49 GB, percent = 43.4%
After initializing ZeRO optimizer
MA 13.4 GB         Max_MA 13.4 GB         CA 13.42 GB         Max_CA 13 GB 
CPU Virtual Memory:  used = 72.5 GB, percent = 43.4%
⏳ 1.0 dk | RAM: 72.30 GB / 167 GB | Durum: Yükleniyor...
⏳ 1.5 dk | RAM: 75.10 GB 

	eval_steps: 100 (from args) != 50 (from trainer_state.json)
	save_steps: 200 (from args) != 100 (from trainer_state.json)


⏳ 18.5 dk | RAM: 123.96 GB / 167 GB | Durum: Yükleniyor...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
1450,0.017200,0.018446
1500,0.017300,0.018387
1550,0.016900,0.018006
1600,0.018600,0.018304
1650,0.017100,0.017741
1700,0.017100,0.017713
1750,0.017200,0.017874
1800,0.018200,0.018732
1850,0.017700,0.018119
1900,0.018100,0.017794


⏳ 19.0 dk | RAM: 124.68 GB / 167 GB | Durum: Yükleniyor...
⏳ 19.5 dk | RAM: 124.65 GB / 167 GB | Durum: Yükleniyor...
⏳ 20.0 dk | RAM: 124.62 GB / 167 GB | Durum: Yükleniyor...
⏳ 20.6 dk | RAM: 124.66 GB / 167 GB | Durum: Yükleniyor...
⏳ 21.1 dk | RAM: 124.66 GB / 167 GB | Durum: Yükleniyor...
⏳ 21.6 dk | RAM: 124.67 GB / 167 GB | Durum: Yükleniyor...
⏳ 22.1 dk | RAM: 124.60 GB / 167 GB | Durum: Yükleniyor...
⏳ 22.6 dk | RAM: 124.69 GB / 167 GB | Durum: Yükleniyor...
⏳ 23.1 dk | RAM: 124.67 GB / 167 GB | Durum: Yükleniyor...
⏳ 23.6 dk | RAM: 124.69 GB / 167 GB | Durum: Yükleniyor...
⏳ 24.1 dk | RAM: 124.68 GB / 167 GB | Durum: Yükleniyor...
⏳ 24.6 dk | RAM: 124.68 GB / 167 GB | Durum: Yükleniyor...
⏳ 25.1 dk | RAM: 124.69 GB / 167 GB | Durum: Yükleniyor...
⏳ 25.6 dk | RAM: 124.68 GB / 167 GB | Durum: Yükleniyor...
⏳ 26.1 dk | RAM: 124.68 GB / 167 GB | Durum: Yükleniyor...
⏳ 26.6 dk | RAM: 124.67 GB / 167 GB | Durum: Yükleniyor...
⏳ 27.1 dk | RAM: 124.69 GB / 167 GB | Durum: Yükleniyor.

In [ ]:
# @title
# Mevcut Pillow'u kaldırıp en güncel ve uyumlu sürümü kuralım
!pip uninstall -y Pillow
!apt-get update && apt-get install -y libaio-dev pdsh
!pip install -q "Pillow>=10.3.0"
!pip install -q -U transformers accelerate deepspeed datasets pandas mpi4py
!pip install -q flash-attn --no-build-isolation

Found existing installation: pillow 12.1.0
Uninstalling pillow-12.1.0:
  Successfully uninstalled pillow-12.1.0
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.sta

In [ ]:
!pip install -q mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.8 MB/s eta 0:00:00


In [ ]:
# Önce mevcut çakışabilecek paketleri temizleyip sonra kuralım
!pip uninstall -y Pillow
!apt-get update && apt-get install -y libaio-dev pdsh
!pip install -q "Pillow>=10.0.0"  # Hatanın ana çözümü
!pip install -q -U transformers accelerate deepspeed datasets pandas mpi4py
!pip install -q flash-attn --no-build-isolation

Found existing installation: pillow 11.3.0
Uninstalling pillow-11.3.0:
  Successfully uninstalled pillow-11.3.0
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://developer.download.nvidia.com/compute/cu

In [ ]:
# @title
# 1. Sistem kütüphaneleri (DeepSpeed'in process yönetimi ve hızlı I/O için)
!apt-get update && apt-get install -y libaio-dev pdsh

# 2. Temel Python kütüphaneleri (En güncel versiyonlar)
!pip install -q -U transformers accelerate deepspeed datasets pillow pandas

# 3. Opsiyonel ama A100 için şiddetle önerilir: Flash Attention 2
# Bu, bellek kullanımını azaltır ve eğitimi ciddi oranda hızlandırır.
!pip install -q flash-attn --no-build-isolation

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
import transformers, accelerate, deepspeed
print(transformers.__version__, accelerate.__version__, deepspeed.__version__)


4.57.3 1.12.0 0.14.5


In [ ]:
!pip -q install -U "transformers==4.46.3" "accelerate==0.34.2" "deepspeed==0.14.5" "datasets" "pillow" "pandas"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 138.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 114.9 MB/s eta 0:00:00


In [ ]:
# @title
# =========================

# QWEN2.5-VL COLAB CELL (FIXED)

# =========================



import os

import time

import json

import glob

from datetime import timedelta

import sys

from functools import lru_cache



import pandas as pd

import torch

from PIL import Image as PImage



from datasets import Dataset

import transformers

from transformers import (

    AutoProcessor,

    BitsAndBytesConfig,

    TrainingArguments,

    Trainer,

    EarlyStoppingCallback,

)



from transformers import Qwen2_5_VLForConditionalGeneration



import peft

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training



# ================== SÜRÜMLERİ YAZDIR ==================



def safe_version(name, obj):

    ver = getattr(obj, "__version__", None)

    print(f"{name} version:", ver if ver is not None else "bulunamadı")



print("==== Python ve Temel Bilgiler ====")

print("Python:", sys.version)

print()



print("==== Ana Kütüphaneler ====")

safe_version("pandas", pd)

safe_version("torch", torch)

safe_version("transformers", transformers)

safe_version("peft", peft)



try:

    import bitsandbytes as bnb

    safe_version("bitsandbytes", bnb)

except ImportError:

    bnb = None

    print("bitsandbytes: yüklü değil (4-bit kapalı çalışacak)")



print("\n==== GPU Durumu ====")

print("torch.cuda.is_available():", torch.cuda.is_available())

if not torch.cuda.is_available():

    print("⚠ GPU yok. Colab: Runtime > Change runtime type > GPU")

    sys.exit(0)

print("GPU:", torch.cuda.get_device_name(0))





# ===================== 1) PATH + PARAMETRELER (COLAB) =====================



# ✅ Tek kök klasör:

# MyDrive/ImageClef-2019-VQA-Med-Training içinde:

# - All_QA_Pairs_train.txt

# - All_QA_Pairs_val.txt

# - Train_images/

# - Val_images/

DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"



TRAIN_QA_FILE = os.path.join(DATA_ROOT, "All_QA_Pairs_train.txt")

VAL_QA_FILE   = os.path.join(DATA_ROOT, "All_QA_Pairs_val.txt")



TRAIN_IMAGE_DIR = os.path.join(DATA_ROOT, "Train_images")

VAL_IMAGE_DIR   = os.path.join(DATA_ROOT, "Val_images")



IMAGE_EXTENSION = os.environ.get("IMAGE_EXTENSION", ".jpg")



OUTPUT_DIR = os.environ.get("OUTPUT_DIR", os.path.join("/content/drive/MyDrive", "qwen_vl_train_output"))

os.makedirs(OUTPUT_DIR, exist_ok=True)



MODEL_ID = os.environ.get("MODEL_ID", "Qwen/Qwen2.5-VL-7B-Instruct")

SYSTEM_PROMPT = os.environ.get("SYSTEM_PROMPT", "You are a medical VQA assistant.")



NUM_TRAIN_EPOCHS = int(os.environ.get("NUM_TRAIN_EPOCHS", "8"))



USE_4BIT = (os.environ.get("USE_4BIT", "1").strip() not in ("0","false","False","no","NO")) and (bnb is not None)

FREEZE_VISION = (os.environ.get("FREEZE_VISION", "1").strip() not in ("0","false","False","no","NO"))

ATTN_IMPL = os.environ.get("ATTN_IMPL", "")  # "flash_attention_2" istersen



print("\n==== PATH KONTROL ====")

print("DATA_ROOT       :", DATA_ROOT)

print("TRAIN_QA_FILE   :", TRAIN_QA_FILE)

print("VAL_QA_FILE     :", VAL_QA_FILE)

print("TRAIN_IMAGE_DIR :", TRAIN_IMAGE_DIR)

print("VAL_IMAGE_DIR   :", VAL_IMAGE_DIR)

print("OUTPUT_DIR      :", OUTPUT_DIR)

print("MODEL_ID        :", MODEL_ID)

print("USE_4BIT        :", USE_4BIT)

print("EPOCHS(max)     :", NUM_TRAIN_EPOCHS)

print("FREEZE_VISION   :", FREEZE_VISION)

print("ATTN_IMPL       :", ATTN_IMPL if ATTN_IMPL else "(default)")



for p in [DATA_ROOT, TRAIN_QA_FILE, VAL_QA_FILE, TRAIN_IMAGE_DIR, VAL_IMAGE_DIR]:

    if not os.path.exists(p):

        raise FileNotFoundError(f"Bulunamadı: {p}")





# ===================== 2) DATA OKUMA =====================



def read_pipe_qa(path_txt: str) -> pd.DataFrame:

    last_err = None

    for enc in ("utf-8", "utf-8-sig", "latin-1"):

        try:

            rows = []

            with open(path_txt, "r", encoding=enc, errors="replace") as f:

                for line in f:

                    line = line.rstrip("\n\r")

                    if not line.strip():

                        continue

                    parts = line.split("|", 2)

                    if len(parts) < 3:

                        continue

                    image_id = parts[0].strip()

                    question = parts[1].strip()

                    answer   = parts[2].strip()

                    rows.append((image_id, question, answer))

            return pd.DataFrame(rows, columns=["image_id", "question", "answer"])

        except Exception as e:

            last_err = e

    raise RuntimeError(f"QA dosyası okunamadı: {path_txt} (son hata: {last_err})")



def load_qa_file(path_txt: str, image_dir: str, name: str) -> pd.DataFrame:

    print(f"\n🔹 {name} TXT okunuyor:", path_txt)



    df = read_pipe_qa(path_txt)

    df["image_id"] = df["image_id"].astype(str).str.strip()

    df["question"] = df["question"].astype(str).str.strip()

    df["answer"]   = df["answer"].astype(str).str.strip()



    df["image_path"] = df["image_id"].apply(lambda x: os.path.join(image_dir, f"{x}{IMAGE_EXTENSION}"))

    df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)



    if len(df) == 0:

        raise RuntimeError(f"{name} için hiç geçerli örnek yok. Klasör/uzantı yanlış olabilir.")



    print(f"{name} QA sayısı     : {len(df)}")

    print(f"{name} unique images : {df['image_id'].nunique()}")

    return df



df_train = load_qa_file(TRAIN_QA_FILE, TRAIN_IMAGE_DIR, "TRAIN")

df_val   = load_qa_file(VAL_QA_FILE,   VAL_IMAGE_DIR,   "VAL")





# ===================== 3) MODEL + PROCESSOR =====================



bnb_config = None

if USE_4BIT:

    bnb_config = BitsAndBytesConfig(

        load_in_4bit=True,

        bnb_4bit_quant_type="nf4",

        bnb_4bit_compute_dtype=torch.float16,

        bnb_4bit_use_double_quant=True,

    )



model_kwargs = dict(

    device_map="auto",

    torch_dtype=torch.float16,

    low_cpu_mem_usage=True,

)

if bnb_config is not None:

    model_kwargs["quantization_config"] = bnb_config

if ATTN_IMPL.strip():

    model_kwargs["attn_implementation"] = ATTN_IMPL.strip()



model = Qwen2_5_VLForConditionalGeneration.from_pretrained(MODEL_ID, **model_kwargs)



processor = AutoProcessor.from_pretrained(MODEL_ID)

tokenizer = processor.tokenizer



# Vision freeze

if FREEZE_VISION:

    if hasattr(model, "visual"):

        for p in model.visual.parameters():

            p.requires_grad = False

        print("✅ model.visual freeze edildi")

    else:

        print("⚠ model.visual bulunamadı; vision freeze atlandı (mimari farklı olabilir).")



if USE_4BIT:

    model = prepare_model_for_kbit_training(model)



# ✅ Stabil (checkpointing kapalı)

model.config.use_cache = True

# model.gradient_checkpointing_enable()   # KAPALI

# model.enable_input_require_grads()      # KAPALI



# LoRA

lora_config = LoraConfig(

    r=int(os.environ.get("LORA_R", "64")),

    lora_alpha=int(os.environ.get("LORA_ALPHA", "128")),

    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],

    lora_dropout=float(os.environ.get("LORA_DROPOUT", "0.02")),

    bias="none",

    task_type="CAUSAL_LM",

)

model = get_peft_model(model, lora_config)



print("\nEğitilebilir Parametre Sayısı:")

model.print_trainable_parameters()





# ===================== 4) PROMPT (Qwen chat template) =====================



def build_messages(question: str):

    return [

        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},

        {"role": "user", "content": [

            {"type": "image"},

            {"type": "text", "text": f"Question: {question}\nAnswer briefly with a short phrase."},

        ]},

    ]



PROMPT_ANSWER_SEP = " "



def format_and_load_dataset(df: pd.DataFrame) -> Dataset:

    prompts, prompt_lens, answers, questions = [], [], [], []

    for _, row in df.iterrows():

        msgs = build_messages(row["question"])



        prompt_text = processor.apply_chat_template(

            msgs,

            tokenize=False,

            add_generation_prompt=True,

        )



        pl = len(tokenizer(prompt_text + PROMPT_ANSWER_SEP, add_special_tokens=True)["input_ids"])



        prompts.append(prompt_text)

        prompt_lens.append(pl)

        answers.append(row["answer"])

        questions.append(row["question"])



    return Dataset.from_dict(

        {

            "image_path": df["image_path"].tolist(),

            "image_id": df["image_id"].tolist(),

            "prompt": prompts,

            "prompt_len": prompt_lens,

            "answer": answers,

            "question": questions,

        }

    )



train_dataset = format_and_load_dataset(df_train)

val_dataset   = format_and_load_dataset(df_val)



print(f"\nEğitim Seti Boyutu : {len(train_dataset)}")

print(f"Validation Seti    : {len(val_dataset)}")





# ===================== 5) IMAGE CACHE =====================



IMG_CACHE_SIZE = int(os.environ.get("IMG_CACHE_SIZE", "4096"))



@lru_cache(maxsize=IMG_CACHE_SIZE)

def _load_rgb_cached(path: str):

    img = PImage.open(path).convert("RGB")

    return img.copy()





# ===================== 6) DATA COLLATOR =====================



def vqa_data_collator(examples):

    images  = [_load_rgb_cached(e["image_path"]) for e in examples]

    prompts = [e["prompt"] for e in examples]

    prompt_lens = [int(e["prompt_len"]) for e in examples]



    eos = tokenizer.eos_token or ""

    answers = [(e["answer"].strip() + (eos if eos else "")).strip() for e in examples]

    full_texts = [p + PROMPT_ANSWER_SEP + a for p, a in zip(prompts, answers)]



    proc_out = processor(

        text=full_texts,

        images=images,

        return_tensors="pt",

        padding=True,

    )



    batch = dict(proc_out)



    input_ids = batch["input_ids"]

    labels = input_ids.clone()



    for i, pl in enumerate(prompt_lens):

        labels[i, :pl] = -100



    if tokenizer.pad_token_id is not None:

        labels[labels == tokenizer.pad_token_id] = -100



    batch["labels"] = labels

    return batch





# ===================== 7) TRAINING ARGS =====================



EVAL_STEPS = int(os.environ.get("EVAL_STEPS", "500"))

SAVE_STEPS = int(os.environ.get("SAVE_STEPS", "1000"))



DL_WORKERS = int(os.environ.get("DL_WORKERS", "2"))

BATCH_SIZE = int(os.environ.get("BATCH_SIZE", "1"))

GRAD_ACCUM = int(os.environ.get("GRAD_ACCUM", "4"))



LR = float(os.environ.get("LR", "3e-5"))

WARMUP_RATIO = float(os.environ.get("WARMUP_RATIO", "0.10"))

SCHED = os.environ.get("SCHED", "constant_with_warmup")

WEIGHT_DECAY = float(os.environ.get("WEIGHT_DECAY", "0.001"))



training_args = TrainingArguments(

    output_dir=OUTPUT_DIR,

    num_train_epochs=NUM_TRAIN_EPOCHS,



    per_device_train_batch_size=BATCH_SIZE,

    gradient_accumulation_steps=GRAD_ACCUM,



    per_device_eval_batch_size=1,

    eval_strategy="steps",

    eval_steps=EVAL_STEPS,



    save_strategy="steps",

    save_steps=SAVE_STEPS,

    save_total_limit=int(os.environ.get("SAVE_TOTAL_LIMIT", "2")),

    save_safetensors=True,



    load_best_model_at_end=True,

    metric_for_best_model="eval_loss",

    greater_is_better=False,



    learning_rate=LR,

    warmup_ratio=WARMUP_RATIO,

    lr_scheduler_type=SCHED,



    weight_decay=WEIGHT_DECAY,

    max_grad_norm=float(os.environ.get("MAX_GRAD_NORM", "1.0")),



    optim=os.environ.get("OPTIM", "adamw_torch"),

    fp16=True,



    logging_steps=int(os.environ.get("LOGGING_STEPS", "50")),

    report_to="none",

    remove_unused_columns=False,



    dataloader_num_workers=DL_WORKERS,

    dataloader_pin_memory=True,

)



PATIENCE = int(os.environ.get("PATIENCE", "3"))

MIN_DELTA = float(os.environ.get("MIN_DELTA", "0.0005"))



trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    eval_dataset=val_dataset,

    data_collator=vqa_data_collator,

    callbacks=[EarlyStoppingCallback(

        early_stopping_patience=PATIENCE,

        early_stopping_threshold=MIN_DELTA

    )],

)



print(f"\n✨ Fine-tuning başlıyor (Qwen2.5-VL, epochs(max)={NUM_TRAIN_EPOCHS}, USE_4BIT={USE_4BIT})")

print(f"OUTPUT_DIR={OUTPUT_DIR}")

print(f"LR={LR}, SCHED={SCHED}, warmup={WARMUP_RATIO}")

print(f"batch={BATCH_SIZE}, grad_accum={GRAD_ACCUM} (effective={BATCH_SIZE*GRAD_ACCUM})")

print(f"eval_steps={EVAL_STEPS}, save_steps={SAVE_STEPS}, patience={PATIENCE}, min_delta={MIN_DELTA}")

print(f"DL_WORKERS={DL_WORKERS}, IMG_CACHE_SIZE={IMG_CACHE_SIZE}")





# ===================== 8) TRAIN + AUTO RESUME =====================



ckpts = sorted(glob.glob(os.path.join(OUTPUT_DIR, "checkpoint-*")))

resume_path = ckpts[-1] if ckpts else None



start_time = time.time()



if resume_path is None:

    print("ℹ️ Checkpoint yok -> sıfırdan başlıyor.")

    train_result = trainer.train()

else:

    print(f"ℹ️ Checkpoint bulundu -> devam: {resume_path}")

    train_result = trainer.train(resume_from_checkpoint=resume_path)



end_time = time.time()

total_time_sec = end_time - start_time



metrics = dict(train_result.metrics)

metrics["total_training_time_sec"] = float(total_time_sec)

metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))

metrics["num_train_samples"] = len(train_dataset)

metrics["num_eval_samples"] = len(val_dataset)

metrics["num_train_steps"] = train_result.global_step



eval_metrics = trainer.evaluate(val_dataset)

for k, v in eval_metrics.items():

    metrics[f"eval_{k}"] = float(v) if isinstance(v, (int, float)) else v



log_obj = {

    "training_args": training_args.to_dict(),

    "final_metrics": metrics,

    "log_history": trainer.state.log_history,

}



log_path = os.path.join(OUTPUT_DIR, "training_log.json")

with open(log_path, "w", encoding="utf-8") as f:

    json.dump(log_obj, f, indent=2)



print(f"\n📄 Log kaydedildi: {log_path}")

print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")



ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_qwen_v1")

trainer.model.save_pretrained(ADAPTER_PATH)

processor.save_pretrained(OUTPUT_DIR)



print(f"\n✅ Eğitim bitti.")

print(f"✅ Adaptör: {ADAPTER_PATH}")

print(f"✅ En iyi model: load_best_model_at_end=True (eval_loss)")

print(f"✅ Checkpointler: {OUTPUT_DIR}/checkpoint-*")





# =========================

# (İSTEĞE BAĞLI) Checkpointing'li alternatif (TAKILMAYAN):

#   model.config.use_cache = False

#   model.gradient_checkpointing_enable()

#   emb = model.get_input_embeddings()

#   emb.weight.requires_grad_(True)

# =========================




==== Python ve Temel Bilgiler ====
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

==== Ana Kütüphaneler ====
pandas version: 2.2.2
torch version: 2.9.0+cu126
transformers version: 4.57.3
peft version: 0.9.0
bitsandbytes version: 0.48.2

==== GPU Durumu ====
torch.cuda.is_available(): True
GPU: NVIDIA A100-SXM4-40GB

==== PATH KONTROL ====
DATA_ROOT       : /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training
TRAIN_QA_FILE   : /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/All_QA_Pairs_train.txt
VAL_QA_FILE     : /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/All_QA_Pairs_val.txt
TRAIN_IMAGE_DIR : /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/Train_images
VAL_IMAGE_DIR   : /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/Val_images
OUTPUT_DIR      : /content/drive/MyDrive/qwen_vl_train_output
MODEL_ID        : Qwen/Qwen2.5-VL-7B-Instruct
USE_4BIT        : True
EPOCHS(max)     : 8
FREEZE_VISION   : True
ATTN_IMPL       : (default)

🔹 TRAIN TX

`torch_dtype` is deprecated! Use `dtype` instead!


VAL QA sayısı     : 2000
VAL unique images : 500


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

...

✅ model.visual freeze edildi

Eğitilebilir Parametre Sayısı:
trainable params: 190,357,504 || all params: 8,482,524,160 || trainable%: 2.2441139030012502

Eğitim Seti Boyutu : 12792
Validation Seti    : 2000

✨ Fine-tuning başlıyor (Qwen2.5-VL, epochs(max)=8, USE_4BIT=True)
OUTPUT_DIR=/content/drive/MyDrive/qwen_vl_train_output
LR=3e-05, SCHED=constant_with_warmup, warmup=0.1
batch=1, grad_accum=4 (effective=4)
eval_steps=500, save_steps=1000, patience=3, min_delta=0.0005
DL_WORKERS=2, IMG_CACHE_SIZE=4096
ℹ️ Checkpoint yok -> sıfırdan başlıyor.


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
500,7.250900,7.262357
1000,7.242900,7.238907
1500,7.184500,7.234898
2000,7.231600,7.231773
2500,7.124400,7.231345
3000,7.194800,7.229610
3500,7.312300,7.228391
4000,7.159500,7.228687
4500,7.206900,7.227803
5000,7.175800,7.227471


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default


📄 Log kaydedildi: /content/drive/MyDrive/qwen_vl_train_output/training_log.json
⏱ Toplam eğitim süresi: 10:21:40

✅ Eğitim bitti.
✅ Adaptör: /content/drive/MyDrive/qwen_vl_train_output/final_adapter_qwen_v1
✅ En iyi model: load_best_model_at_end=True (eval_loss)
✅ Checkpointler: /content/drive/MyDrive/qwen_vl_train_output/checkpoint-*


In [ ]:
# @title
# =========================
# QWEN2.5-VL TEST (TRAINING STYLE – COLAB)
# =========================

import os
import time
import json
import glob
from datetime import timedelta
import sys
from functools import lru_cache
from collections import defaultdict

import pandas as pd
import torch
from PIL import Image as PImage

import transformers
from transformers import (
    AutoProcessor,
    BitsAndBytesConfig,
)

from transformers import Qwen2_5_VLForConditionalGeneration

import peft
from peft import PeftModel


# ================== BASIC INFO ==================

print("Python:", sys.version)
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))


# ===================== PATHS =====================

TRAIN_OUT_DIR = "/content/drive/MyDrive/qwen_vl_train_output"

TEST_TXT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/VQAMed2019_Test_Questions_w_Ref_Answers.txt"
TEST_IMAGE_DIR = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/VQAMed2019_Test_Images"

OUT_DIR = "/content/drive/MyDrive/VQAMed2019Test_results_qwen"
os.makedirs(OUT_DIR, exist_ok=True)

MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct"
IMAGE_EXTENSION = ".jpg"

SYSTEM_PROMPT = "You are a medical VQA assistant."

USE_4BIT = True
MAX_NEW_TOKENS = 16
NUM_BEAMS = 1
LIMIT = 0   # 0 = all


# ===================== LOAD TEST TXT =====================

def load_test_txt(path_txt: str) -> pd.DataFrame:
    rows = []
    with open(path_txt, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split("|")
            if len(parts) < 4:
                continue
            rows.append(
                (
                    parts[0].strip(),  # image_id
                    parts[1].strip(),  # category
                    parts[2].strip(),  # question
                    parts[3].strip(),  # ground_truth
                )
            )
    return pd.DataFrame(rows, columns=["image_id", "category", "question", "ground_truth"])


df = load_test_txt(TEST_TXT)
if LIMIT > 0:
    df = df.head(LIMIT)

print("Test samples:", len(df))


# ===================== FIND ADAPTER =====================

def find_adapter(training_out_dir: str):
    final_adapter = os.path.join(training_out_dir, "final_adapter_qwen_v1")
    if os.path.isdir(final_adapter):
        return final_adapter, "final_adapter_qwen_v1"

    ckpts = sorted(glob.glob(os.path.join(training_out_dir, "checkpoint-*")))
    if not ckpts:
        raise FileNotFoundError("Adapter bulunamadı")
    return ckpts[-1], os.path.basename(ckpts[-1])


adapter_dir, adapter_tag = find_adapter(TRAIN_OUT_DIR)
print("Adapter:", adapter_dir)


# ===================== MODEL + PROCESSOR =====================

bnb_config = None
if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

model_kwargs = dict(
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)
if bnb_config is not None:
    model_kwargs["quantization_config"] = bnb_config

base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    **model_kwargs,
)

processor = AutoProcessor.from_pretrained(MODEL_ID)
tokenizer = processor.tokenizer

model = PeftModel.from_pretrained(base_model, adapter_dir)
model.eval()


# ===================== PROMPT (SAME AS TRAINING) =====================

def build_messages(question: str):
    return [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": f"Question: {question}\nAnswer briefly with a short phrase."},
        ]},
    ]


# ===================== CLEAN OUTPUT =====================

def clean_pred(text: str) -> str:
    if not text:
        return ""
    t = text.strip()
    if "\n" in t:
        t = t.splitlines()[0].strip()
    for bad in ["ASSISTANT:", "assistant:", "Assistant:"]:
        if t.startswith(bad):
            t = t[len(bad):].strip()
    return t[:200]


# ===================== INFERENCE =====================

samples_flat = []
grouped = defaultdict(list)

start = time.time()

for i, row in enumerate(df.itertuples(index=False), start=1):
    img_path = os.path.join(TEST_IMAGE_DIR, f"{row.image_id}{IMAGE_EXTENSION}")

    if not os.path.exists(img_path):
        continue

    image = PImage.open(img_path).convert("RGB")

    msgs = build_messages(row.question)
    prompt_text = processor.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = processor(
        text=prompt_text,
        images=image,
        return_tensors="pt",
        padding=True,
    ).to(model.device)

    input_len = inputs["input_ids"].shape[1]

    gen = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        num_beams=NUM_BEAMS,
        do_sample=False,
    )

    pred_raw = tokenizer.decode(gen[0, input_len:], skip_special_tokens=True)
    pred = clean_pred(pred_raw)

    rec = {
        "index": i,
        "image_id": row.image_id,
        "image_path": img_path,
        "category": row.category,
        "question": row.question,
        "ground_truth": row.ground_truth,
        "predicted": pred,
    }
    samples_flat.append(rec)

    grouped[row.image_id].append({
        "category": row.category,
        "question": row.question,
        "ground_truth": row.ground_truth,
        "predicted": pred,
    })

    if i % 25 == 0:
        elapsed = time.time() - start
        print(f"[{i}/{len(df)}] elapsed={timedelta(seconds=int(elapsed))}")


# ===================== SAVE =====================

stamp = time.strftime("%Y%m%d_%H%M%S")
out_prefix = os.path.join(
    OUT_DIR,
    f"qwen_vl__{adapter_tag}__{stamp}"
)

flat_json = out_prefix + "_preds_flat.json"
grouped_json = out_prefix + "_preds_grouped.json"

with open(flat_json, "w", encoding="utf-8") as f:
    json.dump(
        {
            "total_test_samples": len(samples_flat),
            "samples": samples_flat,
        },
        f,
        indent=2,
        ensure_ascii=False,
    )

with open(grouped_json, "w", encoding="utf-8") as f:
    json.dump(
        {
            "total_test_samples": len(grouped),
            "samples": [
                {
                    "image_id": k,
                    "image_path": os.path.join(TEST_IMAGE_DIR, f"{k}{IMAGE_EXTENSION}"),
                    "qas": v,
                }
                for k, v in grouped.items()
            ],
        },
        f,
        indent=2,
        ensure_ascii=False,
    )

print("\n✅ TEST BİTTİ")
print("Flat   :", flat_json)
print("Grouped:", grouped_json)


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
torch: 2.9.0+cu126
transformers: 4.57.3
peft: 0.9.0
CUDA: True
GPU: NVIDIA A100-SXM4-40GB


`torch_dtype` is deprecated! Use `dtype` instead!


Test samples: 500
Adapter: /content/drive/MyDrive/qwen_vl_train_output/final_adapter_qwen_v1


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[25/500] elapsed=0:00:31
[50/500] elapsed=0:00:58
[75/500] elapsed=0:01:28
[100/500] elapsed=0:01:55
[125/500] elapsed=0:02:20
[150/500] elapsed=0:02:43
[175/500] elapsed=0:03:06
[200/500] elapsed=0:03:29
[225/500] elapsed=0:03:52
[250/500] elapsed=0:04:15
[275/500] elapsed=0:04:44
[300/500] elapsed=0:05:12
[325/500] elapsed=0:05:43
[350/500] elapsed=0:06:13
[375/500] elapsed=0:06:47
[400/500] elapsed=0:07:21
[425/500] elapsed=0:08:01
[450/500] elapsed=0:08:40
[475/500] elapsed=0:09:16
[500/500] elapsed=0:09:56

✅ TEST BİTTİ
Flat   : /content/drive/MyDrive/VQAMed2019Test_results_qwen/qwen_vl__final_adapter_qwen_v1__20260105_091053_preds_flat.json
Grouped: /content/drive/MyDrive/VQAMed2019Test_results_qwen/qwen_vl__final_adapter_qwen_v1__20260105_091053_preds_grouped.json


In [ ]:
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)


In [ ]:
# @title
# =========================
# QWEN2.5-VL BASE TEST (NO TRAINING/NO ADAPTER) – COLAB
# =========================

import os
import time
import json
from datetime import timedelta
import sys
from collections import defaultdict

import pandas as pd
import torch
from PIL import Image as PImage

import transformers
from transformers import AutoProcessor, BitsAndBytesConfig
from transformers import Qwen2_5_VLForConditionalGeneration

# ================== BASIC INFO ==================

print("Python:", sys.version)
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("CUDA:", torch.cuda.is_available())
if not torch.cuda.is_available():
    raise RuntimeError("CUDA yok. Colab: Runtime > Change runtime type > GPU")
print("GPU:", torch.cuda.get_device_name(0))

# ===================== PATHS =====================
# Test dosyalarının olduğu yer (senin verdiğin path mantığıyla)
DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TEST_TXT = os.path.join(DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR = os.path.join(DATA_ROOT, "VQAMed2019_Test_Images")

OUT_DIR = "/content/drive/MyDrive/VQAMed2019Test_results_qwen_BASE"
os.makedirs(OUT_DIR, exist_ok=True)

MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct"
IMAGE_EXTENSION = os.environ.get("IMAGE_EXTENSION", ".jpg")

SYSTEM_PROMPT = "You are a medical VQA assistant."

USE_4BIT = True          # ✅ base model'i 4-bit yükle (VRAM düşer)
MAX_NEW_TOKENS = 16
NUM_BEAMS = 1
LIMIT = 0                # 0 = all

# ===================== LOAD TEST TXT =====================

def load_test_txt(path_txt: str) -> pd.DataFrame:
    rows = []
    with open(path_txt, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split("|")
            if len(parts) < 4:
                continue
            rows.append(
                (
                    parts[0].strip(),  # image_id
                    parts[1].strip(),  # category
                    parts[2].strip(),  # question
                    parts[3].strip(),  # ground_truth
                )
            )
    if not rows:
        raise RuntimeError(f"Test txt boş/okunamadı: {path_txt}")
    return pd.DataFrame(rows, columns=["image_id", "category", "question", "ground_truth"])

if not os.path.exists(TEST_TXT):
    raise FileNotFoundError(f"Bulunamadı: {TEST_TXT}")
if not os.path.isdir(TEST_IMAGE_DIR):
    raise FileNotFoundError(f"Bulunamadı: {TEST_IMAGE_DIR}")

df = load_test_txt(TEST_TXT)
if LIMIT > 0:
    df = df.head(LIMIT)

print("Test samples:", len(df))
print("TEST_TXT:", TEST_TXT)
print("TEST_IMAGE_DIR:", TEST_IMAGE_DIR)
print("OUT_DIR:", OUT_DIR)
print("MODEL_ID:", MODEL_ID)
print("USE_4BIT:", USE_4BIT)

# ===================== MODEL + PROCESSOR (BASE) =====================

bnb_config = None
if USE_4BIT:
    try:
        import bitsandbytes as bnb  # noqa
    except Exception:
        print("⚠ bitsandbytes yok -> USE_4BIT=False'a düşüyorum")
        USE_4BIT = False

if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

model_kwargs = dict(
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)
if bnb_config is not None:
    model_kwargs["quantization_config"] = bnb_config

print("\n🔹 Base model yükleniyor...")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(MODEL_ID, **model_kwargs)
processor = AutoProcessor.from_pretrained(MODEL_ID)
tokenizer = processor.tokenizer

model.eval()

# ===================== PROMPT (SAME AS TRAINING) =====================

def build_messages(question: str):
    return [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": f"Question: {question}\nAnswer briefly with a short phrase."},
        ]},
    ]

def clean_pred(text: str) -> str:
    if not text:
        return ""
    t = text.strip()
    if "\n" in t:
        t = t.splitlines()[0].strip()
    for bad in ["ASSISTANT:", "assistant:", "Assistant:"]:
        if t.startswith(bad):
            t = t[len(bad):].strip()
    return t[:200]

# ===================== INFERENCE =====================

samples_flat = []
grouped = defaultdict(list)

start = time.time()
total = len(df)

for idx, row in enumerate(df.itertuples(index=False), start=1):
    img_path = os.path.join(TEST_IMAGE_DIR, f"{row.image_id}{IMAGE_EXTENSION}")

    if not os.path.exists(img_path):
        # kaybolan image varsa yine de kayıt düşelim
        samples_flat.append({
            "index": idx,
            "image_id": row.image_id,
            "image_path": img_path,
            "category": row.category,
            "question": row.question,
            "ground_truth": row.ground_truth,
            "predicted": "",
            "error": "image_not_found",
        })
        continue

    image = PImage.open(img_path).convert("RGB")

    msgs = build_messages(row.question)
    prompt_text = processor.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = processor(
        text=prompt_text,
        images=image,
        return_tensors="pt",
        padding=True,
    ).to(model.device)

    input_len = inputs["input_ids"].shape[1]

    with torch.inference_mode():
        gen = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            num_beams=NUM_BEAMS,
            do_sample=False,
        )

    pred_raw = tokenizer.decode(gen[0, input_len:], skip_special_tokens=True)
    pred = clean_pred(pred_raw)

    rec = {
        "index": idx,
        "image_id": row.image_id,
        "image_path": img_path,
        "category": row.category,
        "question": row.question,
        "ground_truth": row.ground_truth,
        "predicted": pred,
    }
    samples_flat.append(rec)

    grouped[row.image_id].append({
        "category": row.category,
        "question": row.question,
        "ground_truth": row.ground_truth,
        "predicted": pred,
    })

    if idx % 25 == 0:
        elapsed = time.time() - start
        print(f"[{idx}/{total}] elapsed={timedelta(seconds=int(elapsed))}")

inf_sec = time.time() - start

# ===================== SAVE =====================

stamp = time.strftime("%Y%m%d_%H%M%S")
out_prefix = os.path.join(OUT_DIR, f"qwen_BASE__{stamp}")

flat_json = out_prefix + "_preds_flat.json"
grouped_json = out_prefix + "_preds_grouped.json"

with open(flat_json, "w", encoding="utf-8") as f:
    json.dump(
        {
            "total_test_samples": int(len(samples_flat)),
            "inference_time_sec": float(inf_sec),
            "inference_time_str": str(timedelta(seconds=int(inf_sec))),
            "model_id": MODEL_ID,
            "use_4bit": bool(USE_4BIT),
            "samples": samples_flat,
        },
        f,
        indent=2,
        ensure_ascii=False,
    )

with open(grouped_json, "w", encoding="utf-8") as f:
    json.dump(
        {
            "total_test_samples": int(len(grouped)),
            "inference_time_sec": float(inf_sec),
            "inference_time_str": str(timedelta(seconds=int(inf_sec))),
            "model_id": MODEL_ID,
            "use_4bit": bool(USE_4BIT),
            "samples": [
                {
                    "image_id": k,
                    "image_path": os.path.join(TEST_IMAGE_DIR, f"{k}{IMAGE_EXTENSION}"),
                    "qas": v,
                }
                for k, v in grouped.items()
            ],
        },
        f,
        indent=2,
        ensure_ascii=False,
    )

print("\n✅ BASE TEST BİTTİ")
print("Flat   :", flat_json)
print("Grouped:", grouped_json)


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
torch: 2.9.0+cu126
transformers: 4.57.3
CUDA: True
GPU: NVIDIA A100-SXM4-40GB
Test samples: 500
TEST_TXT: /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/VQAMed2019_Test_Questions_w_Ref_Answers.txt
TEST_IMAGE_DIR: /content/drive/MyDrive/ImageClef-2019-VQA-Med-Training/VQAMed2019_Test_Images
OUT_DIR: /content/drive/MyDrive/VQAMed2019Test_results_qwen_BASE
MODEL_ID: Qwen/Qwen2.5-VL-7B-Instruct
USE_4BIT: True


`torch_dtype` is deprecated! Use `dtype` instead!



🔹 Base model yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[25/500] elapsed=0:00:12
[50/500] elapsed=0:00:24
[75/500] elapsed=0:00:36
[100/500] elapsed=0:00:48
[125/500] elapsed=0:00:59
[150/500] elapsed=0:01:12
[175/500] elapsed=0:01:25
[200/500] elapsed=0:01:38
[225/500] elapsed=0:01:51
[250/500] elapsed=0:02:03
[275/500] elapsed=0:02:15
[300/500] elapsed=0:02:27
[325/500] elapsed=0:02:40
[350/500] elapsed=0:02:51
[375/500] elapsed=0:03:06
[400/500] elapsed=0:03:22
[425/500] elapsed=0:03:38
[450/500] elapsed=0:03:53
[475/500] elapsed=0:04:08
[500/500] elapsed=0:04:23

✅ BASE TEST BİTTİ
Flat   : /content/drive/MyDrive/VQAMed2019Test_results_qwen_BASE/qwen_BASE__20260105_095037_preds_flat.json
Grouped: /content/drive/MyDrive/VQAMed2019Test_results_qwen_BASE/qwen_BASE__20260105_095037_preds_grouped.json


In [ ]:
import os
import json
import time
import glob
import torch
import pandas as pd
from PIL import Image as PImage
from datetime import timedelta
from collections import defaultdict
from transformers import LlavaForConditionalGeneration, LlavaProcessor

# ===================== AYARLAR =====================

# 1. Eğitim çıktılarının olduğu ana klasör
TRAIN_OUT_DIR = "/content/drive/MyDrive/llava_fullft_zero2_out"

# 2. VQA-Med 2019 Test Verileri (Senin dizin yapın)
DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"
TEST_TXT = os.path.join(DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR = os.path.join(DATA_ROOT, "VQAMed2019_Test_Images")

# 3. Sonuçların kaydedileceği klasör
RESULTS_DIR = os.path.join(TRAIN_OUT_DIR, "test_results_final")
os.makedirs(RESULTS_DIR, exist_ok=True)

# 4. Model Bilgileri (Processor hatasını önlemek için ana model ID gerekli)
BASE_MODEL_ID = "llava-hf/llava-1.5-7b-hf"
MAX_NEW_TOKENS = 32
IMAGE_EXTENSION = ".jpg"

# ===================== FONKSİYONLAR =====================

def find_latest_checkpoint(base_dir):
    """Dizindeki en yüksek numaralı checkpoint klasörünü bulur."""
    checkpoints = glob.glob(os.path.join(base_dir, "checkpoint-*"))
    if not checkpoints:
        # Eğer klasörler doğrudan ana dizindeyse kontrol et
        if os.path.exists(os.path.join(base_dir, "model.safetensors.index.json")):
            return base_dir
        return None
    checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
    return checkpoints[-1]

def build_prompt(question):
    """Eğitimde kullanılan prompt formatının aynısı."""
    return f"USER: <image>\nQuestion: {question}\nAnswer briefly.\nASSISTANT:"

def clean_pred(text):
    """Model çıktısını temizler."""
    if not text: return ""
    t = text.strip()
    # Eğer model çok uzun cevap verirse sadece ilk satırı al
    if "\n" in t: t = t.splitlines()[0].strip()
    return t

# ===================== TEST DÖNGÜSÜ =====================

@torch.inference_mode()
def run_llava_test():
    # 1. Checkpoint Belirleme
    latest_ckpt = find_latest_checkpoint(TRAIN_OUT_DIR)
    if not latest_ckpt:
        print("❌ HATA: Checkpoint klasörü bulunamadı! Lütfen TRAIN_OUT_DIR yolunu kontrol edin.")
        return

    print(f"🔄 Model Ağırlıkları Yükleniyor: {latest_ckpt}")

    # 2. Model ve Processor Yükleme
    # A100 için optimize: bfloat16 ve auto device_map
    model = LlavaForConditionalGeneration.from_pretrained(
        latest_ckpt,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        low_cpu_mem_usage=True
    )

    # KRİTİK: Checkpoint'te eksik olabilen preprocessor_config için ana modelden yüklüyoruz
    print(f"🔄 Processor Yükleniyor: {BASE_MODEL_ID}")
    processor = LlavaProcessor.from_pretrained(BASE_MODEL_ID)
    model.eval()

    # 3. Test Verilerini Yükleme
    if not os.path.exists(TEST_TXT):
        print(f"❌ HATA: Test dosyası bulunamadı: {TEST_TXT}")
        return

    rows = []
    with open(TEST_TXT, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("|")
            if len(parts) >= 4:
                rows.append({
                    "image_id": parts[0].strip(),
                    "category": parts[1].strip(),
                    "question": parts[2].strip(),
                    "ground_truth": parts[3].strip()
                })

    df = pd.DataFrame(rows)
    print(f"📦 Toplam {len(df)} test örneği işlenecek.")

    # 4. Tahmin Döngüsü
    samples_flat = []
    grouped = defaultdict(list)
    start_time = time.time()

    for idx, row in enumerate(df.itertuples(), start=1):
        img_path = os.path.join(TEST_IMAGE_DIR, f"{row.image_id}{IMAGE_EXTENSION}")

        if not os.path.exists(img_path):
            continue

        # Görüntü hazırlama
        image = PImage.open(img_path).convert("RGB")
        prompt = build_prompt(row.question)

        # Tokenize ve Inference
        inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device, torch.bfloat16)
        input_len = inputs["input_ids"].shape[1]

        output_ids = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            num_beams=1
        )

        # Decode
        decoded = processor.batch_decode(output_ids[:, input_len:], skip_special_tokens=True)[0]
        prediction = clean_pred(decoded)

        # Veri Kaydı
        res_item = {
            "index": idx,
            "image_id": row.image_id,
            "category": row.category,
            "question": row.question,
            "ground_truth": row.ground_truth,
            "predicted": prediction
        }
        samples_flat.append(res_item)
        grouped[row.image_id].append(res_item)

        if idx % 50 == 0:
            elapsed = time.time() - start_time
            print(f"进度: [{idx}/{len(df)}] | Geçen Süre: {timedelta(seconds=int(elapsed))}")

    # 5. SONUÇLARI KAYDETME
    inf_sec = time.time() - start_time
    stamp = time.strftime("%Y%m%d_%H%M")
    ckpt_name = os.path.basename(latest_ckpt)

    # A) Flat JSON (Sıralı Liste)
    flat_path = os.path.join(RESULTS_DIR, f"results_{ckpt_name}_{stamp}_flat.json")
    with open(flat_path, "w", encoding="utf-8") as f:
        json.dump({
            "checkpoint": latest_ckpt,
            "inference_time": str(timedelta(seconds=int(inf_sec))),
            "samples": samples_flat
        }, f, indent=4, ensure_ascii=False)

    # B) Grouped JSON (Resim Bazlı Gruplanmış)
    grouped_path = os.path.join(RESULTS_DIR, f"results_{ckpt_name}_{stamp}_grouped.json")
    with open(grouped_path, "w", encoding="utf-8") as f:
        json.dump({
            "checkpoint": latest_ckpt,
            "total_images": len(grouped),
            "images": [
                {"image_id": k, "qas": v} for k, v in grouped.items()
            ]
        }, f, indent=4, ensure_ascii=False)

    print(f"\n✅ TEST BİTTİ!")
    print(f"📊 Flat Sonuçlar: {flat_path}")
    print(f"📊 Gruplanmış Sonuçlar: {grouped_path}")

if __name__ == "__main__":
    run_llava_test()

🔄 Model Ağırlıkları Yükleniyor: /content/drive/MyDrive/llava_fullft_zero2_out/checkpoint-700


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

🔄 Processor Yükleniyor: llava-hf/llava-1.5-7b-hf


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

📦 Toplam 500 test örneği işlenecek.
进度: [50/500] | Geçen Süre: 0:00:54
进度: [100/500] | Geçen Süre: 0:01:46
进度: [150/500] | Geçen Süre: 0:02:35
进度: [200/500] | Geçen Süre: 0:03:23
进度: [250/500] | Geçen Süre: 0:04:11
进度: [300/500] | Geçen Süre: 0:05:04
进度: [350/500] | Geçen Süre: 0:05:55
进度: [400/500] | Geçen Süre: 0:06:48
进度: [450/500] | Geçen Süre: 0:07:43
进度: [500/500] | Geçen Süre: 0:08:40

✅ TEST BİTTİ!
📊 Flat Sonuçlar: /content/drive/MyDrive/llava_fullft_zero2_out/test_results_final/results_checkpoint-700_20260110_1420_flat.json
📊 Gruplanmış Sonuçlar: /content/drive/MyDrive/llava_fullft_zero2_out/test_results_final/results_checkpoint-700_20260110_1420_grouped.json


In [ ]:
!pip uninstall -y bitsandbytes
!pip install --no-cache-dir bitsandbytes==0.48.2

In [ ]:
!pip install -q \
    evaluate \
    sacrebleu \
    rouge-score \
    bert-score \
    rapidfuzz \
    nltk \
    sentencepiece \
    accelerate

In [ ]:
# @title
import os
import time
import json
from datetime import timedelta
import sys

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    pipeline,
)

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# bitsandbytes varsa sürümünü de yazalım
try:
    import bitsandbytes as bnb
except ImportError:
    bnb = None

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

if bnb is not None:
    safe_version("bitsandbytes", bnb)
else:
    print("bitsandbytes: yüklü değil")

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ===================== 1. PATH ve PARAMETRELER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

# 🔴 BURAYI DEĞİŞTİREREK C1 / C2 / C3 / C4 ARASINDA GEÇİŞ YAPABİLİRSİN
QA_FILE_PATH = os.path.join(LOCAL_DATA_ROOT, "C4_Abnormality_train.txt")

IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "Train_images")
OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_3epoch_colab")

TEST_RATIO = 0.15          # 600'ün %15'i test olacak
NUM_TRAIN_EPOCHS = 3
IMAGE_EXTENSION = ".jpg"

# 🔹 SADECE 600 ÖRNEK KULLAN
TOTAL_SAMPLE_COUNT = 600

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nKullanılan cihaz: {device}")
if device.type != "cuda":
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yapmayı unutma.")
    sys.exit(0)

if not os.path.exists(IMAGE_DIR):
    raise FileNotFoundError(f"Görsel klasörü bulunamadı: {IMAGE_DIR}")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Çıktılar bu klasöre kaydedilecek: {OUTPUT_DIR}")

# ===================== 2. DATAFRAME -> HF DATASET =====================

def format_and_load_dataset(dataframe: pd.DataFrame) -> Dataset:
    """
    Görselleri burada RAM'e yüklemiyoruz, sadece path + text tutuyoruz.
    Asıl encoding, Trainer içindeki data_collator'da batch bazında yapılacak.
    """
    texts = []
    for _, row in dataframe.iterrows():
        prompt = (
            f"SYSTEM: {SYSTEM_PROMPT}\n"
            f"USER: <image>\n{row['question']}\n"
            f"ASSISTANT: {row['answer']}"
        )
        texts.append(prompt)

    return Dataset.from_dict(
        {
            "image_path": dataframe["image_path"].tolist(),
            "text": texts,
            "question": dataframe["question"].tolist(),
            "true_answer": dataframe["answer"].tolist(),
        }
    )

# TXT dosyasını oku
df_qa = pd.read_csv(
    QA_FILE_PATH,
    sep="|",
    header=None,
    names=["image_id", "question", "answer"],
    encoding="latin-1",
)
df_qa["image_id"] = df_qa["image_id"].str.strip()
df_qa["question"] = df_qa["question"].str.strip()
df_qa["answer"] = df_qa["answer"].str.strip()

# Görsel path'lerini oluştur
df_qa["image_path"] = df_qa["image_id"].apply(
    lambda x: os.path.join(IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

# Sadece gerçekten var olan görselleri tut
df_qa = df_qa[df_qa["image_path"].apply(os.path.exists)].reset_index(drop=True)

if len(df_qa) == 0:
    raise RuntimeError("Hiç geçerli örnek bulunamadı, image_path'ler yanlış olabilir.")

# 🔹 SADECE 600 ÖRNEK SEÇ
if len(df_qa) < TOTAL_SAMPLE_COUNT:
    print(f"⚠ Uyarı: Veri setinde {len(df_qa)} örnek var, 600'den az. Hepsi kullanılacak.")
    df_sampled = df_qa
else:
    df_sampled = df_qa.sample(n=TOTAL_SAMPLE_COUNT, random_state=42).reset_index(drop=True)

# Shuffle et
df_shuffled = df_sampled.sample(frac=1.0, random_state=123).reset_index(drop=True)

# 600 * 0.15 = 90 test, 510 train
test_count = int(len(df_shuffled) * TEST_RATIO)
train_count = len(df_shuffled) - test_count

df_train = df_shuffled.iloc[:train_count]
df_test = df_shuffled.iloc[train_count:]

train_dataset = format_and_load_dataset(df_train)
test_dataset = format_and_load_dataset(df_test)

print(f"\nToplam seçilen örnek sayısı: {len(df_shuffled)}")
print(f"Eğitim Seti Boyutu: {len(train_dataset)} (Train)")
print(f"Test Seti Boyutu: {len(test_dataset)} (Test, %{TEST_RATIO * 100:.0f})")
print(f"Kullanılan SYSTEM PROMPT: {SYSTEM_PROMPT}")

# ===================== 3. MODEL + QLoRA KURULUMU =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",   # VRAM yetiyorsa sabit {"": 0} da deneyebilirsin
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Vision encoder'ı dondur
for param in model.vision_tower.parameters():
    param.requires_grad = False

# 4-bit için modeli hazırla
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

processor = LlavaProcessor.from_pretrained(MODEL_ID)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4. DATA COLLATOR (RAM Dostu) =====================

def vqa_data_collator(examples):
    """
    examples: [{"image_path": ..., "text": ..., "question": ..., "true_answer": ...}, ...]
    Her batch için:
      - Görselleri diskten okur
      - Prompt'ları alır
      - LLaVA processor ile token + pixel_values üretir
    """
    images = [PImage.open(e["image_path"]).convert("RGB") for e in examples]
    texts = [e["text"] for e in examples]

    proc_out = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding="longest",   # truncation yok -> image token mismatch yok
    )

    input_ids = proc_out["input_ids"]
    attention_mask = proc_out["attention_mask"]
    pixel_values = proc_out["pixel_values"]

    labels = input_ids.clone()

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels,
    }
    return batch

# ===================== 5. TRAININGARGUMENTS + TRAINER =====================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,

    # senin sürümde evaluation_strategy yok, eval_strategy var
    eval_strategy="epoch",      # her epoch sonunda eval
    save_strategy="epoch",      # her epoch sonunda checkpoint

    logging_steps=10,
    learning_rate=2e-4,
    optim="adamw_torch",
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=vqa_data_collator,
)

print("\n✨ Fine-Tuning Başlıyor (3 Epoch, 600 örnek)...")

# ===================== 6. EĞİTİM + LOG DOSYASI =====================

start_time = time.time()
train_result = trainer.train()
end_time = time.time()
total_time_sec = end_time - start_time

# Train metrikleri
metrics = dict(train_result.metrics)
metrics["total_training_time_sec"] = float(total_time_sec)
metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))
metrics["num_train_samples"] = len(train_dataset)
metrics["num_eval_samples"] = len(test_dataset)
metrics["num_train_steps"] = train_result.global_step

# Eval metrikleri (loss vs.) – test set üzerinde
eval_metrics = trainer.evaluate(test_dataset)
for k, v in eval_metrics.items():
    if isinstance(v, (int, float)):
        metrics[f"eval_{k}"] = float(v)
    else:
        metrics[f"eval_{k}"] = v

# Trainer'ın log history'si (adım adım loss, lr vs.)
log_history = trainer.state.log_history

# Her şeyi tek JSON objesinde topla
log_obj = {
    "training_args": training_args.to_dict(),
    "final_metrics": metrics,
    "log_history": log_history,   # step/epoch bazlı loss + eval_loss kayıtları
}

log_path = os.path.join(OUTPUT_DIR, "training_log_600.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log dosyası kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

# Model ve processor kaydet
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_600")
trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Eğitim tamamlandı ve adaptörler '{ADAPTER_PATH}' konumuna kaydedildi.")

# ===================== 7. BASİT TEST (BİRKAç ÖRNEK) =====================

print("\n📊 Test setinden birkaç örnek üzerinde inference deneniyor...")

vqa_pipe = pipeline(
    "visual-question-answering",
    model=model,
    tokenizer=processor.tokenizer,
    image_processor=processor.image_processor,
    device=0,
)

num_examples_to_show = 3
for i in range(min(num_examples_to_show, len(test_dataset))):
    sample = test_dataset[i]
    question = sample["question"]
    true_answer = sample["true_answer"]

    img = PImage.open(sample["image_path"]).convert("RGB")
    result = vqa_pipe(img, question, max_new_tokens=20)
    pred = result[0]["generated_text"].strip()

    print(f"\n[{i+1}] Soru: {question}")
    print(f"    GT : {true_answer}")
    print(f"    Pred: {pred}")


In [ ]:
# @title
import os
import time
import json
from datetime import timedelta
import sys

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    pipeline,
)

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# bitsandbytes varsa sürümünü de yazalım
try:
    import bitsandbytes as bnb
except ImportError:
    bnb = None

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

if bnb is not None:
    safe_version("bitsandbytes", bnb)
else:
    print("bitsandbytes: yüklü değil")

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ===================== 1. PATH ve PARAMETRELER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

# 🔴 BURAYI DEĞİŞTİREREK C1 / C2 / C3 / C4 ARASINDA GEÇİŞ YAPABİLİRSİN
QA_FILE_PATH = os.path.join(LOCAL_DATA_ROOT, "C4_Abnormality_train.txt")

IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "Train_images")
OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_3epoch_colab")

TEST_RATIO = 0.15          # 600'ün %15'i test olacak
NUM_TRAIN_EPOCHS = 3
IMAGE_EXTENSION = ".jpg"

# 🔹 SADECE 600 ÖRNEK KULLAN
TOTAL_SAMPLE_COUNT = 600

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nKullanılan cihaz: {device}")
if device.type != "cuda":
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yapmayı unutma.")
    sys.exit(0)

if not os.path.exists(IMAGE_DIR):
    raise FileNotFoundError(f"Görsel klasörü bulunamadı: {IMAGE_DIR}")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Çıktılar bu klasöre kaydedilecek: {OUTPUT_DIR}")

# ===================== 2. DATAFRAME -> HF DATASET =====================

def format_and_load_dataset(dataframe: pd.DataFrame) -> Dataset:
    """
    Görselleri burada RAM'e yüklemiyoruz, sadece path + text tutuyoruz.
    Asıl encoding, Trainer içindeki data_collator'da batch bazında yapılacak.
    """
    texts = []
    for _, row in dataframe.iterrows():
        prompt = (
            f"SYSTEM: {SYSTEM_PROMPT}\n"
            f"USER: <image>\n{row['question']}\n"
            f"ASSISTANT: {row['answer']}"
        )
        texts.append(prompt)

    return Dataset.from_dict(
        {
            "image_path": dataframe["image_path"].tolist(),
            "text": texts,
            "question": dataframe["question"].tolist(),
            "true_answer": dataframe["answer"].tolist(),
        }
    )

# TXT dosyasını oku
df_qa = pd.read_csv(
    QA_FILE_PATH,
    sep="|",
    header=None,
    names=["image_id", "question", "answer"],
    encoding="latin-1",
)
df_qa["image_id"] = df_qa["image_id"].str.strip()
df_qa["question"] = df_qa["question"].str.strip()
df_qa["answer"] = df_qa["answer"].str.strip()

# Görsel path'lerini oluştur
df_qa["image_path"] = df_qa["image_id"].apply(
    lambda x: os.path.join(IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

# Sadece gerçekten var olan görselleri tut
df_qa = df_qa[df_qa["image_path"].apply(os.path.exists)].reset_index(drop=True)

if len(df_qa) == 0:
    raise RuntimeError("Hiç geçerli örnek bulunamadı, image_path'ler yanlış olabilir.")

# 🔹 SADECE 600 ÖRNEK SEÇ
if len(df_qa) < TOTAL_SAMPLE_COUNT:
    print(f"⚠ Uyarı: Veri setinde {len(df_qa)} örnek var, 600'den az. Hepsi kullanılacak.")
    df_sampled = df_qa
else:
    df_sampled = df_qa.sample(n=TOTAL_SAMPLE_COUNT, random_state=42).reset_index(drop=True)

# Shuffle et
df_shuffled = df_sampled.sample(frac=1.0, random_state=123).reset_index(drop=True)

# 600 * 0.15 = 90 test, 510 train
test_count = int(len(df_shuffled) * TEST_RATIO)
train_count = len(df_shuffled) - test_count

df_train = df_shuffled.iloc[:train_count]
df_test = df_shuffled.iloc[train_count:]

train_dataset = format_and_load_dataset(df_train)
test_dataset = format_and_load_dataset(df_test)

print(f"\nToplam seçilen örnek sayısı: {len(df_shuffled)}")
print(f"Eğitim Seti Boyutu: {len(train_dataset)} (Train)")
print(f"Test Seti Boyutu: {len(test_dataset)} (Test, %{TEST_RATIO * 100:.0f})")
print(f"Kullanılan SYSTEM PROMPT: {SYSTEM_PROMPT}")

# ===================== 3. MODEL + QLoRA KURULUMU =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",   # VRAM yetiyorsa sabit {"": 0} da deneyebilirsin
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Vision encoder'ı dondur
for param in model.vision_tower.parameters():
    param.requires_grad = False

# 4-bit için modeli hazırla
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

processor = LlavaProcessor.from_pretrained(MODEL_ID)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4. DATA COLLATOR (RAM Dostu) =====================

def vqa_data_collator(examples):
    """
    examples: [{"image_path": ..., "text": ..., "question": ..., "true_answer": ...}, ...]
    Her batch için:
      - Görselleri diskten okur
      - Prompt'ları alır
      - LLaVA processor ile token + pixel_values üretir
    """
    images = [PImage.open(e["image_path"]).convert("RGB") for e in examples]
    texts = [e["text"] for e in examples]

    proc_out = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding="longest",   # truncation yok -> image token mismatch yok
    )

    input_ids = proc_out["input_ids"]
    attention_mask = proc_out["attention_mask"]
    pixel_values = proc_out["pixel_values"]

    labels = input_ids.clone()

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels,
    }
    return batch

# ===================== 5. TRAININGARGUMENTS + TRAINER =====================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,

    # senin sürümde evaluation_strategy yok, eval_strategy var
    eval_strategy="epoch",      # her epoch sonunda eval
    save_strategy="epoch",      # her epoch sonunda checkpoint

    logging_steps=10,
    learning_rate=2e-4,
    optim="adamw_torch",
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=vqa_data_collator,
)

print("\n✨ Fine-Tuning Başlıyor (3 Epoch, 600 örnek)...")

# ===================== 6. EĞİTİM + LOG DOSYASI =====================

start_time = time.time()
train_result = trainer.train()
end_time = time.time()
total_time_sec = end_time - start_time

# Train metrikleri
metrics = dict(train_result.metrics)
metrics["total_training_time_sec"] = float(total_time_sec)
metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))
metrics["num_train_samples"] = len(train_dataset)
metrics["num_eval_samples"] = len(test_dataset)
metrics["num_train_steps"] = train_result.global_step

# Eval metrikleri (loss vs.) – test set üzerinde
eval_metrics = trainer.evaluate(test_dataset)
for k, v in eval_metrics.items():
    if isinstance(v, (int, float)):
        metrics[f"eval_{k}"] = float(v)
    else:
        metrics[f"eval_{k}"] = v

# Trainer'ın log history'si (adım adım loss, lr vs.)
log_history = trainer.state.log_history

# Her şeyi tek JSON objesinde topla
log_obj = {
    "training_args": training_args.to_dict(),
    "final_metrics": metrics,
    "log_history": log_history,   # step/epoch bazlı loss + eval_loss kayıtları
}

log_path = os.path.join(OUTPUT_DIR, "training_log_600.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log dosyası kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

# Model ve processor kaydet
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_600")
trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Eğitim tamamlandı ve adaptörler '{ADAPTER_PATH}' konumuna kaydedildi.")

# ===================== 7. BASİT TEST (BİRKAç ÖRNEK) =====================

print("\n📊 Test setinden birkaç örnek üzerinde inference deneniyor...")

vqa_pipe = pipeline(
    "visual-question-answering",
    model=model,
    tokenizer=processor.tokenizer,
    image_processor=processor.image_processor,
    device=0,
)

num_examples_to_show = 3
for i in range(min(num_examples_to_show, len(test_dataset))):
    sample = test_dataset[i]
    question = sample["question"]
    true_answer = sample["true_answer"]

    img = PImage.open(sample["image_path"]).convert("RGB")
    result = vqa_pipe(img, question, max_new_tokens=20)
    pred = result[0]["generated_text"].strip()

    print(f"\n[{i+1}] Soru: {question}")
    print(f"    GT : {true_answer}")
    print(f"    Pred: {pred}")


# ===================== 8. TÜM TEST SETİ SONUÇLARINI KAYDET =====================

print("\n📄 Tüm test seti prediction sonuçları dosyaya kaydediliyor...")

all_results = []

correct = 0

for idx in range(len(test_dataset)):
    sample = test_dataset[idx]

    img = PImage.open(sample["image_path"]).convert("RGB")
    question = sample["question"]
    true_answer = sample["true_answer"]

    pred = vqa_pipe(img, question, max_new_tokens=20)[0]["generated_text"].strip()

    is_match = (pred.lower().strip() == true_answer.lower().strip())
    if is_match:
        correct += 1

    all_results.append({
        "index": idx + 1,
        "question": question,
        "ground_truth": true_answer,
        "predicted": pred,
        "match": is_match
    })

# Accuracy hesapla
accuracy = (correct / len(test_dataset)) * 100

# JSON objesi oluştur
test_output = {
    "total_test_samples": len(test_dataset),
    "accuracy_exact_match_percent": accuracy,
    "samples": all_results
}

# Kaydedilecek dosya yolu
test_log_path = os.path.join(OUTPUT_DIR, "test_predictions.json")

with open(test_log_path, "w", encoding="utf-8") as f:
    json.dump(test_output, f, indent=2, ensure_ascii=False)

print(f"📁 Test prediction dosyası kaydedildi: {test_log_path}")
print(f"🎯 Exact Match Accuracy: {accuracy:.2f}%")


In [ ]:
# @title
import os
import time
import json
from datetime import timedelta
import sys
import re
import string

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Global metrikler
import evaluate
from rapidfuzz import fuzz

# bitsandbytes varsa sürümünü de yazalım
try:
    import bitsandbytes as bnb
except ImportError:
    bnb = None

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

if bnb is not None:
    safe_version("bitsandbytes", bnb)
else:
    print("bitsandbytes: yüklü değil")

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ===================== 1. PATH ve PARAMETRELER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

# 🔴 Artık C3 Organ için:
QA_FILE_PATH = os.path.join(LOCAL_DATA_ROOT, "C3_Organ_train.txt")

IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "Train_images")
OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "c3_fine_tune")

TEST_RATIO = 0.15          # 1200'ün %15'i test olacak
NUM_TRAIN_EPOCHS = 3
IMAGE_EXTENSION = ".jpg"

# 🔹 SADECE 1200 ÖRNEK KULLAN
TOTAL_SAMPLE_COUNT = 1200

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nKullanılan cihaz: {device}")
if device.type != "cuda":
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yapmayı unutma.")
    sys.exit(0)

if not os.path.exists(IMAGE_DIR):
    raise FileNotFoundError(f"Görsel klasörü bulunamadı: {IMAGE_DIR}")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Çıktılar bu klasöre kaydedilecek: {OUTPUT_DIR}")

# ===================== 2. DATAFRAME -> HF DATASET =====================

def format_and_load_dataset(dataframe: pd.DataFrame) -> Dataset:
    """
    Görselleri burada RAM'e yüklemiyoruz, sadece path + text tutuyoruz.
    Asıl encoding, Trainer içindeki data_collator'da batch bazında yapılacak.
    """
    texts = []
    for _, row in dataframe.iterrows():
        prompt = (
            f"SYSTEM: {SYSTEM_PROMPT}\n"
            f"USER: <image>\n{row['question']}\n"
            f"ASSISTANT: {row['answer']}"
        )
        texts.append(prompt)

    return Dataset.from_dict(
        {
            "image_path": dataframe["image_path"].tolist(),
            "text": texts,
            "question": dataframe["question"].tolist(),
            "true_answer": dataframe["answer"].tolist(),
        }
    )

print("\n🔹 TXT dosyası okunuyor:", QA_FILE_PATH)

# TXT dosyasını oku
df_qa = pd.read_csv(
    QA_FILE_PATH,
    sep="|",
    header=None,
    names=["image_id", "question", "answer"],
    encoding="latin-1",
)
df_qa["image_id"] = df_qa["image_id"].str.strip()
df_qa["question"] = df_qa["question"].str.strip()
df_qa["answer"] = df_qa["answer"].str.strip()

# Görsel path'lerini oluştur
df_qa["image_path"] = df_qa["image_id"].apply(
    lambda x: os.path.join(IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

# Sadece gerçekten var olan görselleri tut
df_qa = df_qa[df_qa["image_path"].apply(os.path.exists)].reset_index(drop=True)

if len(df_qa) == 0:
    raise RuntimeError("Hiç geçerli örnek bulunamadı, image_path'ler yanlış olabilir.")

# 🔹 SADECE 1200 ÖRNEK SEÇ
if len(df_qa) < TOTAL_SAMPLE_COUNT:
    print(f"⚠ Uyarı: Veri setinde {len(df_qa)} örnek var, 1200'den az. Hepsi kullanılacak.")
    df_sampled = df_qa
else:
    df_sampled = df_qa.sample(n=TOTAL_SAMPLE_COUNT, random_state=42).reset_index(drop=True)

# Shuffle et
df_shuffled = df_sampled.sample(frac=1.0, random_state=123).reset_index(drop=True)

test_count = int(len(df_shuffled) * TEST_RATIO)
train_count = len(df_shuffled) - test_count

df_train = df_shuffled.iloc[:train_count]
df_test = df_shuffled.iloc[train_count:]

train_dataset = format_and_load_dataset(df_train)
test_dataset = format_and_load_dataset(df_test)

print(f"\nToplam seçilen örnek sayısı: {len(df_shuffled)}")
print(f"Eğitim Seti Boyutu: {len(train_dataset)} (Train)")
print(f"Test Seti Boyutu: {len(test_dataset)} (Test, %{TEST_RATIO * 100:.0f})")
print(f"Kullanılan SYSTEM PROMPT: {SYSTEM_PROMPT}")

# ===================== 3. MODEL + QLoRA KURULUMU =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",   # VRAM yetiyorsa sabit {"": 0} da deneyebilirsin
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Vision encoder'ı dondur
for param in model.vision_tower.parameters():
    param.requires_grad = False

# 4-bit için modeli hazırla
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

processor = LlavaProcessor.from_pretrained(MODEL_ID)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4. DATA COLLATOR (RAM Dostu) =====================

def vqa_data_collator(examples):
    """
    examples: [{"image_path": ..., "text": ..., "question": ..., "true_answer": ...}, ...]
    Her batch için:
      - Görselleri diskten okur
      - Prompt'ları alır
      - LLaVA processor ile token + pixel_values üretir
    """
    images = [PImage.open(e["image_path"]).convert("RGB") for e in examples]
    texts = [e["text"] for e in examples]

    proc_out = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding="longest",   # truncation yok -> image token mismatch yok
    )

    input_ids = proc_out["input_ids"]
    attention_mask = proc_out["attention_mask"]
    pixel_values = proc_out["pixel_values"]

    labels = input_ids.clone()

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels,
    }
    return batch

# ===================== 5. TRAININGARGUMENTS + TRAINER =====================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,

    eval_strategy="epoch",      # her epoch sonunda eval
    save_strategy="epoch",      # her epoch sonunda checkpoint

    logging_steps=10,
    learning_rate=2e-4,
    optim="adamw_torch",
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=vqa_data_collator,
)

print("\n✨ Fine-Tuning Başlıyor (3 Epoch, 1200 örnek, C3 Organ)...")

# ===================== 6. EĞİTİM + LOG DOSYASI =====================

start_time = time.time()
train_result = trainer.train()
end_time = time.time()
total_time_sec = end_time - start_time

# Train metrikleri
metrics = dict(train_result.metrics)
metrics["total_training_time_sec"] = float(total_time_sec)
metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))
metrics["num_train_samples"] = len(train_dataset)
metrics["num_eval_samples"] = len(test_dataset)
metrics["num_train_steps"] = train_result.global_step

# Eval metrikleri (loss vs.) – test set üzerinde
eval_metrics = trainer.evaluate(test_dataset)
for k, v in eval_metrics.items():
    if isinstance(v, (int, float)):
        metrics[f"eval_{k}"] = float(v)
    else:
        metrics[f"eval_{k}"] = v

# Trainer'ın log history'si (adım adım loss, lr vs.)
log_history = trainer.state.log_history

# Her şeyi tek JSON objesinde topla
log_obj = {
    "training_args": training_args.to_dict(),
    "final_metrics": metrics,
    "log_history": log_history,   # step/epoch bazlı loss + eval_loss kayıtları
}

log_path = os.path.join(OUTPUT_DIR, "training_log_c3_organ_1200.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log dosyası kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

# Model ve processor kaydet
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_c3_organ_1200")
trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Eğitim tamamlandı ve adaptörler '{ADAPTER_PATH}' konumuna kaydedildi.")

# ===================== 7. BAZI ÖRNEKLER ÜZERİNDE MANUEL INFERENCE =====================

print("\n📊 Test setinden birkaç örnek üzerinde inference (Manuel Yöntem):")

model.eval()
model.to(device)

num_examples_to_show = 3
for i in range(min(num_examples_to_show, len(test_dataset))):
    sample = test_dataset[i]
    question = sample["question"]
    true_answer = sample["true_answer"]

    img = PImage.open(sample["image_path"]).convert("RGB")

    prompt_text = f"USER: <image>\n{question}\nASSISTANT:"

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )[0].strip()

    print(f"\n[{i+1}] Soru : {question}")
    print(f"     GT   : {true_answer}")
    print(f"     Pred : {pred}")

# ===================== 8. TÜM TEST SETİ İÇİN PREDICTION + GLOBAL METRİKLER =====================

print("\n📄 Tüm test seti için prediction + global metrikler hesaplanıyor (Manuel Yöntem)...")

all_results = []
correct_strict = 0

start_inf = time.time()

for idx in range(len(test_dataset)):
    sample = test_dataset[idx]

    img = PImage.open(sample["image_path"]).convert("RGB")
    question = sample["question"]
    true_answer = sample["true_answer"]

    prompt_text = f"USER: <image>\n{question}\nASSISTANT:"

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )[0].strip()

    is_match = (pred.lower().strip() == true_answer.lower().strip())
    if is_match:
        correct_strict += 1

    if idx % 50 == 0 and idx > 0:
        print(f"  > {idx}/{len(test_dataset)} örnek tamamlandı.")

    all_results.append({
        "index": idx + 1,
        "question": question,
        "ground_truth": true_answer,
        "predicted": pred,
        "match_strict": is_match
    })

end_inf = time.time()
total_inf_sec = end_inf - start_inf

n = len(test_dataset)
accuracy_strict = (correct_strict / n) * 100 if n > 0 else 0.0

# ---------- Global metrikler ----------

pred_texts = [r["predicted"] for r in all_results]
gt_texts = [r["ground_truth"] for r in all_results]

def normalize_answer(s: str) -> str:
    """SQuAD tarzı normalize: lower, punctuation, articles, extra space."""
    def lower(text):
        return text.lower()

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    return white_space_fix(remove_articles(remove_punc(lower(s))))

normalized_matches = 0
for p, g in zip(pred_texts, gt_texts):
    if normalize_answer(p) == normalize_answer(g):
        normalized_matches += 1

accuracy_normalized = (normalized_matches / n) * 100 if n > 0 else 0.0

# Fuzzy
fuzzy_scores_partial = [
    fuzz.partial_ratio(p.lower(), g.lower()) for p, g in zip(pred_texts, gt_texts)
]
fuzzy_acc_80 = sum(s >= 80 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
fuzzy_acc_90 = sum(s >= 90 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
avg_fuzzy_score = sum(fuzzy_scores_partial) / n if n > 0 else 0.0

print("\n📏 Global metrikler hesaplanıyor (BLEU, ROUGE, METEOR, BERTScore)...")

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

bleu_res = bleu_metric.compute(
    predictions=pred_texts,
    references=[[g] for g in gt_texts]
)
rouge_res = rouge_metric.compute(
    predictions=pred_texts,
    references=gt_texts
)
meteor_res = meteor_metric.compute(
    predictions=pred_texts,
    references=gt_texts
)
bertscore_res = bertscore_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
    lang="en"
)

avg_bertscore_f1 = float(sum(bertscore_res["f1"]) / len(bertscore_res["f1"])) if n > 0 else 0.0

print("\n==== METRİK ÖZETİ (C3 Organ, 1200 örnek) ====")
print(f"Strict Exact Match Accuracy     : {accuracy_strict:.2f}%")
print(f"Normalized Exact Match Accuracy : {accuracy_normalized:.2f}%")
print(f"Fuzzy Partial Acc (>=80)        : {fuzzy_acc_80:.2f}%")
print(f"Fuzzy Partial Acc (>=90)        : {fuzzy_acc_90:.2f}%")
print(f"Avg Fuzzy Partial Score         : {avg_fuzzy_score:.2f}")
print(f"BLEU                            : {bleu_res['bleu']:.4f}")
print(f"ROUGE-L                         : {rouge_res['rougeL']:.4f}")
print(f"METEOR                          : {meteor_res['meteor']:.4f}")
print(f"BERTScore Avg F1                : {avg_bertscore_f1:.4f}")

# ---------- JSON çıkışı ----------

test_output = {
    "total_test_samples": n,
    "metrics": {
        "exact_match_strict_percent": accuracy_strict,
        "exact_match_normalized_percent": accuracy_normalized,
        "fuzzy_partial_acc_80_percent": fuzzy_acc_80,
        "fuzzy_partial_acc_90_percent": fuzzy_acc_90,
        "fuzzy_partial_avg_score": avg_fuzzy_score,
        "bleu": float(bleu_res["bleu"]),
        "rougeL": float(rouge_res["rougeL"]),
        "meteor": float(meteor_res["meteor"]),
        "bertscore_avg_f1": avg_bertscore_f1,
    },
    "inference_time_sec": float(total_inf_sec),
    "inference_time_str": str(timedelta(seconds=int(total_inf_sec))),
    "samples": all_results,
}

test_log_path = os.path.join(OUTPUT_DIR, "test_predictions_with_metrics_c3_organ_1200.json")

with open(test_log_path, "w", encoding="utf-8") as f:
    json.dump(test_output, f, indent=2, ensure_ascii=False)

print(f"\n📁 Test prediction + metrikler kaydedildi: {test_log_path}")
print("\n✅ İşlem tamamlandı (C3 Organ).")



In [ ]:
# @title
import os
import time
import json
from datetime import timedelta
import sys
import re
import string

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Global metrikler
import evaluate
from rapidfuzz import fuzz

# bitsandbytes varsa sürümünü de yazalım
try:
    import bitsandbytes as bnb
except ImportError:
    bnb = None

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

if bnb is not None:
    safe_version("bitsandbytes", bnb)
else:
    print("bitsandbytes: yüklü değil")

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

# ===================== 1. PATH ve PARAMETRELER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

# 🔴 Artık C4 Abnormality için:
QA_FILE_PATH = os.path.join(LOCAL_DATA_ROOT, "C4_Abnormality_train.txt")

IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "Train_images")
OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_2epoch_colab_C4_abnormality")

TEST_RATIO = 0.15          # 900'ün %15'i test olacak
NUM_TRAIN_EPOCHS = 2
IMAGE_EXTENSION = ".jpg"

# 🔹 SADECE 900 ÖRNEK KULLAN
TOTAL_SAMPLE_COUNT = 900

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nKullanılan cihaz: {device}")
if device.type != "cuda":
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yapmayı unutma.")
    sys.exit(0)

if not os.path.exists(IMAGE_DIR):
    raise FileNotFoundError(f"Görsel klasörü bulunamadı: {IMAGE_DIR}")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Çıktılar bu klasöre kaydedilecek: {OUTPUT_DIR}")

# ===================== 2. DATAFRAME -> HF DATASET =====================

def format_and_load_dataset(dataframe: pd.DataFrame) -> Dataset:
    """
    Görselleri burada RAM'e yüklemiyoruz, sadece path + text tutuyoruz.
    Asıl encoding, Trainer içindeki data_collator'da batch bazında yapılacak.
    """
    texts = []
    for _, row in dataframe.iterrows():
        prompt = (
            f"SYSTEM: {SYSTEM_PROMPT}\n"
            f"USER: <image>\n{row['question']}\n"
            f"ASSISTANT: {row['answer']}"
        )
        texts.append(prompt)

    return Dataset.from_dict(
        {
            "image_path": dataframe["image_path"].tolist(),
            "text": texts,
            "question": dataframe["question"].tolist(),
            "true_answer": dataframe["answer"].tolist(),
        }
    )

print("\n🔹 TXT dosyası okunuyor:", QA_FILE_PATH)

# TXT dosyasını oku
df_qa = pd.read_csv(
    QA_FILE_PATH,
    sep="|",
    header=None,
    names=["image_id", "question", "answer"],
    encoding="latin-1",
)
df_qa["image_id"] = df_qa["image_id"].str.strip()
df_qa["question"] = df_qa["question"].str.strip()
df_qa["answer"] = df_qa["answer"].str.strip()

# Görsel path'lerini oluştur
df_qa["image_path"] = df_qa["image_id"].apply(
    lambda x: os.path.join(IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

# Sadece gerçekten var olan görselleri tut
df_qa = df_qa[df_qa["image_path"].apply(os.path.exists)].reset_index(drop=True)

if len(df_qa) == 0:
    raise RuntimeError("Hiç geçerli örnek bulunamadı, image_path'ler yanlış olabilir.")

# 🔹 SADECE 900 ÖRNEK SEÇ
if len(df_qa) < TOTAL_SAMPLE_COUNT:
    print(f"⚠ Uyarı: Veri setinde {len(df_qa)} örnek var, 900'den az. Hepsi kullanılacak.")
    df_sampled = df_qa
else:
    df_sampled = df_qa.sample(n=TOTAL_SAMPLE_COUNT, random_state=42).reset_index(drop=True)

# Shuffle et
df_shuffled = df_sampled.sample(frac=1.0, random_state=123).reset_index(drop=True)

test_count = int(len(df_shuffled) * TEST_RATIO)
train_count = len(df_shuffled) - test_count

df_train = df_shuffled.iloc[:train_count]
df_test = df_shuffled.iloc[train_count:]

train_dataset = format_and_load_dataset(df_train)
test_dataset = format_and_load_dataset(df_test)

print(f"\nToplam seçilen örnek sayısı: {len(df_shuffled)}")
print(f"Eğitim Seti Boyutu: {len(train_dataset)} (Train)")
print(f"Test Seti Boyutu: {len(test_dataset)} (Test, %{TEST_RATIO * 100:.0f})")
print(f"Kullanılan SYSTEM PROMPT: {SYSTEM_PROMPT}")

# ===================== 3. MODEL + QLoRA KURULUMU =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",   # VRAM yetiyorsa sabit {"": 0} da deneyebilirsin
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Vision encoder'ı dondur
for param in model.vision_tower.parameters():
    param.requires_grad = False

# 4-bit için modeli hazırla
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

processor = LlavaProcessor.from_pretrained(MODEL_ID)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4. DATA COLLATOR (RAM Dostu) =====================

def vqa_data_collator(examples):
    """
    examples: [{"image_path": ..., "text": ..., "question": ..., "true_answer": ...}, ...]
    Her batch için:
      - Görselleri diskten okur
      - Prompt'ları alır
      - LLaVA processor ile token + pixel_values üretir
    """
    images = [PImage.open(e["image_path"]).convert("RGB") for e in examples]
    texts = [e["text"] for e in examples]

    proc_out = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding="longest",   # truncation yok -> image token mismatch yok
    )

    input_ids = proc_out["input_ids"]
    attention_mask = proc_out["attention_mask"]
    pixel_values = proc_out["pixel_values"]

    labels = input_ids.clone()

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels,
    }
    return batch

# ===================== 5. TRAININGARGUMENTS + TRAINER =====================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,

    # senin sürümde evaluation_strategy yok, eval_strategy var
    eval_strategy="epoch",      # her epoch sonunda eval
    save_strategy="epoch",      # her epoch sonunda checkpoint

    logging_steps=10,
    learning_rate=2e-4,
    optim="adamw_torch",
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=vqa_data_collator,
)

print("\n✨ Fine-Tuning Başlıyor (2 Epoch, 900 örnek, C4 Abnormality)...")

# ===================== 6. EĞİTİM + LOG DOSYASI =====================

start_time = time.time()
train_result = trainer.train()
end_time = time.time()
total_time_sec = end_time - start_time

# Train metrikleri
metrics = dict(train_result.metrics)
metrics["total_training_time_sec"] = float(total_time_sec)
metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))
metrics["num_train_samples"] = len(train_dataset)
metrics["num_eval_samples"] = len(test_dataset)
metrics["num_train_steps"] = train_result.global_step

# Eval metrikleri (loss vs.) – test set üzerinde
eval_metrics = trainer.evaluate(test_dataset)
for k, v in eval_metrics.items():
    if isinstance(v, (int, float)):
        metrics[f"eval_{k}"] = float(v)
    else:
        metrics[f"eval_{k}"] = v

# Trainer'ın log history'si (adım adım loss, lr vs.)
log_history = trainer.state.log_history

# Her şeyi tek JSON objesinde topla
log_obj = {
    "training_args": training_args.to_dict(),
    "final_metrics": metrics,
    "log_history": log_history,   # step/epoch bazlı loss + eval_loss kayıtları
}

log_path = os.path.join(OUTPUT_DIR, "training_log_C4_abnormality_900.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log dosyası kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

# Model ve processor kaydet
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_C4_abnormality_900")
trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Eğitim tamamlandı ve adaptörler '{ADAPTER_PATH}' konumuna kaydedildi.")

# ===================== 7. BAZI ÖRNEKLER ÜZERİNDE MANUEL INFERENCE =====================

print("\n📊 Test setinden birkaç örnek üzerinde inference (Manuel Yöntem):")

model.eval()
model.to(device)

num_examples_to_show = 3
for i in range(min(num_examples_to_show, len(test_dataset))):
    sample = test_dataset[i]
    question = sample["question"]
    true_answer = sample["true_answer"]

    img = PImage.open(sample["image_path"]).convert("RGB")

    prompt_text = f"USER: <image>\n{question}\nASSISTANT:"

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )[0].strip()

    print(f"\n[{i+1}] Soru : {question}")
    print(f"     GT   : {true_answer}")
    print(f"     Pred : {pred}")

# ===================== 8. TÜM TEST SETİ İÇİN PREDICTION + GLOBAL METRİKLER =====================

print("\n📄 Tüm test seti için prediction + global metrikler hesaplanıyor (Manuel Yöntem)...")

all_results = []
correct_strict = 0

start_inf = time.time()

for idx in range(len(test_dataset)):
    sample = test_dataset[idx]

    img = PImage.open(sample["image_path"]).convert("RGB")
    question = sample["question"]
    true_answer = sample["true_answer"]

    prompt_text = f"USER: <image>\n{question}\nASSISTANT:"

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )[0].strip()

    is_match = (pred.lower().strip() == true_answer.lower().strip())
    if is_match:
        correct_strict += 1

    if idx % 50 == 0 and idx > 0:
        print(f"  > {idx}/{len(test_dataset)} örnek tamamlandı.")

    all_results.append({
        "index": idx + 1,
        "question": question,
        "ground_truth": true_answer,
        "predicted": pred,
        "match_strict": is_match
    })

end_inf = time.time()
total_inf_sec = end_inf - start_inf

n = len(test_dataset)
accuracy_strict = (correct_strict / n) * 100 if n > 0 else 0.0

# ---------- Global metrikler ----------

pred_texts = [r["predicted"] for r in all_results]
gt_texts = [r["ground_truth"] for r in all_results]

def normalize_answer(s: str) -> str:
    """SQuAD tarzı normalize: lower, punctuation, articles, extra space."""
    def lower(text):
        return text.lower()

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    return white_space_fix(remove_articles(remove_punc(lower(s))))

normalized_matches = 0
for p, g in zip(pred_texts, gt_texts):
    if normalize_answer(p) == normalize_answer(g):
        normalized_matches += 1

accuracy_normalized = (normalized_matches / n) * 100 if n > 0 else 0.0

# Fuzzy
fuzzy_scores_partial = [
    fuzz.partial_ratio(p.lower(), g.lower()) for p, g in zip(pred_texts, gt_texts)
]
fuzzy_acc_80 = sum(s >= 80 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
fuzzy_acc_90 = sum(s >= 90 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
avg_fuzzy_score = sum(fuzzy_scores_partial) / n if n > 0 else 0.0

print("\n📏 Global metrikler hesaplanıyor (BLEU, ROUGE, METEOR, BERTScore)...")

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

bleu_res = bleu_metric.compute(
    predictions=pred_texts,
    references=[[g] for g in gt_texts]
)
rouge_res = rouge_metric.compute(
    predictions=pred_texts,
    references=gt_texts
)
meteor_res = meteor_metric.compute(
    predictions=pred_texts,
    references=gt_texts
)
bertscore_res = bertscore_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
    lang="en"
)

avg_bertscore_f1 = float(sum(bertscore_res["f1"]) / len(bertscore_res["f1"])) if n > 0 else 0.0

print("\n==== METRİK ÖZETİ (C4 Abnormality, 900 örnek) ====")
print(f"Strict Exact Match Accuracy     : {accuracy_strict:.2f}%")
print(f"Normalized Exact Match Accuracy : {accuracy_normalized:.2f}%")
print(f"Fuzzy Partial Acc (>=80)        : {fuzzy_acc_80:.2f}%")
print(f"Fuzzy Partial Acc (>=90)        : {fuzzy_acc_90:.2f}%")
print(f"Avg Fuzzy Partial Score         : {avg_fuzzy_score:.2f}")
print(f"BLEU                            : {bleu_res['bleu']:.4f}")
print(f"ROUGE-L                         : {rouge_res['rougeL']:.4f}")
print(f"METEOR                          : {meteor_res['meteor']:.4f}")
print(f"BERTScore Avg F1                : {avg_bertscore_f1:.4f}")

# ---------- JSON çıkışı ----------

test_output = {
    "total_test_samples": n,
    "metrics": {
        "exact_match_strict_percent": accuracy_strict,
        "exact_match_normalized_percent": accuracy_normalized,
        "fuzzy_partial_acc_80_percent": fuzzy_acc_80,
        "fuzzy_partial_acc_90_percent": fuzzy_acc_90,
        "fuzzy_partial_avg_score": avg_fuzzy_score,
        "bleu": float(bleu_res["bleu"]),
        "rougeL": float(rouge_res["rougeL"]),
        "meteor": float(meteor_res["meteor"]),
        "bertscore_avg_f1": avg_bertscore_f1,
    },
    "inference_time_sec": float(total_inf_sec),
    "inference_time_str": str(timedelta(seconds=int(total_inf_sec))),
    "samples": all_results,
}

test_log_path = os.path.join(OUTPUT_DIR, "test_predictions_with_metrics_C4_abnormality.json")

with open(test_log_path, "w", encoding="utf-8") as f:
    json.dump(test_output, f, indent=2, ensure_ascii=False)

print(f"\n📁 Test prediction + metrikler kaydedildi: {test_log_path}")
print("\n✅ İşlem tamamlandı (C4 Abnormality).")

In [ ]:
# @title
import os
import time
import json
from datetime import timedelta
import sys
import re
import string

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

try:
    import bitsandbytes as bnb
    safe_version("bitsandbytes", bnb)
except ImportError:
    bnb = None
    print("bitsandbytes: yüklü değil")

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yapmayı unutma.")
    sys.exit(0)

device = torch.device("cuda")
print(f"Kullanılan cihaz: {device}")

# ===================== 1. PATH ve PARAMETRELER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

# 🔹 Train ve Validation TXT dosyaları
TRAIN_QA_FILE = os.path.join(LOCAL_DATA_ROOT, "All_QA_Pairs_train.txt")
VAL_QA_FILE   = os.path.join(LOCAL_DATA_ROOT, "All_QA_Pairs_val.txt")

# 🔹 Görsel klasörleri
TRAIN_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "Train_images")
VAL_IMAGE_DIR   = os.path.join(LOCAL_DATA_ROOT, "Val_images")  # senin dediğin klasör adı

IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_3epoch")
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Çıktılar bu klasöre kaydedilecek: {OUTPUT_DIR}")

NUM_TRAIN_EPOCHS = 4   # compute'u umursamadan makul kalite hedefi

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

# ===================== 2. DATAFRAME -> HF DATASET =====================

def format_and_load_dataset(df: pd.DataFrame) -> Dataset:
    texts = []
    for _, row in df.iterrows():
        prompt = (
            f"SYSTEM: {SYSTEM_PROMPT}\n"
            f"USER: <image>\n{row['question']}\n"
            f"ASSISTANT: {row['answer']}"
        )
        texts.append(prompt)

    return Dataset.from_dict(
        {
            "image_path": df["image_path"].tolist(),
            "image_id": df["image_id"].tolist(),
            "text": texts,
            "question": df["question"].tolist(),
            "true_answer": df["answer"].tolist(),
        }
    )

def load_qa_file(path_txt: str, image_dir: str, name: str) -> pd.DataFrame:
    print(f"\n🔹 {name} TXT okunuyor:", path_txt)

    df = pd.read_csv(
        path_txt,
        sep="|",
        header=None,
        names=["image_id", "question", "answer"],
        encoding="latin-1",
    )

    df["image_id"] = df["image_id"].str.strip()
    df["question"] = df["question"].str.strip()
    df["answer"]   = df["answer"].str.strip()

    df["image_path"] = df["image_id"].apply(
        lambda x: os.path.join(image_dir, f"{x}{IMAGE_EXTENSION}")
    )

    # sadece path'i gerçekten var olan görseller
    df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

    if len(df) == 0:
        raise RuntimeError(f"{name} için hiç geçerli örnek bulunamadı, {image_dir} veya uzantı yanlış olabilir.")

    print(f"{name} QA sayısı        : {len(df)}")
    print(f"{name} eşsiz image_id    : {df['image_id'].nunique()}")
    return df

# Train ve Val dataframes
df_train = load_qa_file(TRAIN_QA_FILE, TRAIN_IMAGE_DIR, "TRAIN")
df_val   = load_qa_file(VAL_QA_FILE,   VAL_IMAGE_DIR,   "VAL")

train_dataset = format_and_load_dataset(df_train)
val_dataset   = format_and_load_dataset(df_val)

print(f"\nEğitim Seti Boyutu : {len(train_dataset)}")
print(f"Validation Seti    : {len(val_dataset)}")

# ===================== 3. MODEL + QLoRA KURULUMU =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Vision encoder'ı dondur
for param in model.vision_tower.parameters():
    param.requires_grad = False

# 4-bit için modeli hazırla
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

processor = LlavaProcessor.from_pretrained(MODEL_ID)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4. DATA COLLATOR =====================

def vqa_data_collator(examples):
    images = [PImage.open(e["image_path"]).convert("RGB") for e in examples]
    texts  = [e["text"] for e in examples]

    proc_out = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding="longest",
    )

    input_ids      = proc_out["input_ids"]
    attention_mask = proc_out["attention_mask"]
    pixel_values   = proc_out["pixel_values"]

    labels = input_ids.clone()

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels,
    }
    return batch

# ===================== 5. TRAININGARGUMENTS + TRAINER =====================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,

    eval_strategy="epoch",      # her epoch sonunda val_loss
    save_strategy="epoch",      # her epoch sonunda checkpoint

    logging_steps=50,
    learning_rate=2e-4,
    optim="adamw_torch",
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=vqa_data_collator,
)

print(f"\n✨ Fine-tuning başlıyor (All train QA, ayrı VAL set, {NUM_TRAIN_EPOCHS} epoch)...")

# ===================== 6. EĞİTİM + LOG DOSYASI =====================

start_time = time.time()
train_result = trainer.train()
end_time = time.time()
total_time_sec = end_time - start_time

metrics = dict(train_result.metrics)
metrics["total_training_time_sec"] = float(total_time_sec)
metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))
metrics["num_train_samples"] = len(train_dataset)
metrics["num_eval_samples"] = len(val_dataset)
metrics["num_train_steps"] = train_result.global_step

eval_metrics = trainer.evaluate(val_dataset)
for k, v in eval_metrics.items():
    if isinstance(v, (int, float)):
        metrics[f"eval_{k}"] = float(v)
    else:
        metrics[f"eval_{k}"] = v

log_history = trainer.state.log_history

log_obj = {
    "training_args": training_args.to_dict(),
    "final_metrics": metrics,
    "log_history": log_history,
}

log_path = os.path.join(OUTPUT_DIR, "training_log_allQA_with_val_3epoch.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log dosyası kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

# Model ve processor kaydet
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_3epoch")
trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Eğitim tamamlandı. Adaptörler '{ADAPTER_PATH}' klasörüne kaydedildi.")


In [ ]:
# @title
# DEVAM ETTİRME KODU



# ================== 0. COLAB DRIVE MOUNT + IMPORTLAR ==================
from google.colab import drive
drive.mount('/content/drive')

import os
import time
import json
from datetime import timedelta
import sys
import re
import string
import glob

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

try:
    import bitsandbytes as bnb
    safe_version("bitsandbytes", bnb)
except ImportError:
    bnb = None
    print("bitsandbytes: yüklü değil")

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yapmayı unutma.")
    sys.exit(0)

device = torch.device("cuda")
print(f"Kullanılan cihaz: {device}")

# ===================== 1. PATH ve PARAMETRELER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

# 🔹 Train ve Validation TXT dosyaları
TRAIN_QA_FILE = os.path.join(LOCAL_DATA_ROOT, "All_QA_Pairs_train.txt")
VAL_QA_FILE   = os.path.join(LOCAL_DATA_ROOT, "All_QA_Pairs_val.txt")

# 🔹 Görsel klasörleri
TRAIN_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "Train_images")
VAL_IMAGE_DIR   = os.path.join(LOCAL_DATA_ROOT, "Val_images")  # senin dediğin klasör adı

IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_3epoch")
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Çıktılar bu klasöre kaydedilecek: {OUTPUT_DIR}")

NUM_TRAIN_EPOCHS = 4   # compute'u umursamadan makul kalite hedefi

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

# ===================== 2. DATAFRAME -> HF DATASET =====================

def format_and_load_dataset(df: pd.DataFrame) -> Dataset:
    texts = []
    for _, row in df.iterrows():
        prompt = (
            f"SYSTEM: {SYSTEM_PROMPT}\n"
            f"USER: <image>\n{row['question']}\n"
            f"ASSISTANT: {row['answer']}"
        )
        texts.append(prompt)

    return Dataset.from_dict(
        {
            "image_path": df["image_path"].tolist(),
            "image_id": df["image_id"].tolist(),
            "text": texts,
            "question": df["question"].tolist(),
            "true_answer": df["answer"].tolist(),
        }
    )

def load_qa_file(path_txt: str, image_dir: str, name: str) -> pd.DataFrame:
    print(f"\n🔹 {name} TXT okunuyor:", path_txt)

    df = pd.read_csv(
        path_txt,
        sep="|",
        header=None,
        names=["image_id", "question", "answer"],
        encoding="latin-1",
    )

    df["image_id"] = df["image_id"].str.strip()
    df["question"] = df["question"].str.strip()
    df["answer"]   = df["answer"].str.strip()

    df["image_path"] = df["image_id"].apply(
        lambda x: os.path.join(image_dir, f"{x}{IMAGE_EXTENSION}")
    )

    # sadece path'i gerçekten var olan görseller
    df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

    if len(df) == 0:
        raise RuntimeError(f"{name} için hiç geçerli örnek bulunamadı, {image_dir} veya uzantı yanlış olabilir.")

    print(f"{name} QA sayısı        : {len(df)}")
    print(f"{name} eşsiz image_id    : {df['image_id'].nunique()}")
    return df

# Train ve Val dataframes
df_train = load_qa_file(TRAIN_QA_FILE, TRAIN_IMAGE_DIR, "TRAIN")
df_val   = load_qa_file(VAL_QA_FILE,   VAL_IMAGE_DIR,   "VAL")

train_dataset = format_and_load_dataset(df_train)
val_dataset   = format_and_load_dataset(df_val)

print(f"\nEğitim Seti Boyutu : {len(train_dataset)}")
print(f"Validation Seti    : {len(val_dataset)}")

# ===================== 3. MODEL + QLoRA KURULUMU =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Vision encoder'ı dondur
for param in model.vision_tower.parameters():
    param.requires_grad = False

# 4-bit için modeli hazırla
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

processor = LlavaProcessor.from_pretrained(MODEL_ID)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4. DATA COLLATOR =====================

def vqa_data_collator(examples):
    images = [PImage.open(e["image_path"]).convert("RGB") for e in examples]
    texts  = [e["text"] for e in examples]

    proc_out = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding="longest",
    )

    input_ids      = proc_out["input_ids"]
    attention_mask = proc_out["attention_mask"]
    pixel_values   = proc_out["pixel_values"]

    labels = input_ids.clone()

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels,
    }
    return batch

# ===================== 5. TRAININGARGUMENTS + TRAINER =====================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,

    eval_strategy="epoch",      # HF bazı versiyonlarda evaluation_strategy ile aynı
    save_strategy="epoch",      # her epoch sonunda checkpoint

    logging_steps=50,
    learning_rate=2e-4,
    optim="adamw_torch",
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=vqa_data_collator,
)

# ===================== 6. CHECKPOINT BUL VE DEVAM ET =====================

# Var olan checkpoint'leri bul (ör: checkpoint-12792, checkpoint-25584, checkpoint-38376)
ckpts = sorted(
    glob.glob(os.path.join(OUTPUT_DIR, "checkpoint-*")),
    key=lambda p: int(p.split("-")[-1])
)

if ckpts:
    print("\nBulunan checkpoint'ler:")
    for c in ckpts:
        print("  ", c)
    last_ckpt = ckpts[-1]
    print(f"\n✅ Son checkpoint'ten devam edilecek: {last_ckpt}")
else:
    last_ckpt = None
    print("\nℹ Checkpoint bulunamadı, eğitim sıfırdan başlayacak.")

print(f"\n✨ Fine-tuning başlıyor (All train QA, ayrı VAL set, {NUM_TRAIN_EPOCHS} epoch)...")

start_time = time.time()

if last_ckpt is not None:
    train_result = trainer.train(resume_from_checkpoint=last_ckpt)
else:
    train_result = trainer.train()

end_time = time.time()
total_time_sec = end_time - start_time

# ===================== 7. METRİKLER + LOG KAYDI =====================

metrics = dict(train_result.metrics)
metrics["total_training_time_sec"] = float(total_time_sec)
metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))
metrics["num_train_samples"] = len(train_dataset)
metrics["num_eval_samples"] = len(val_dataset)
metrics["num_train_steps"] = train_result.global_step

eval_metrics = trainer.evaluate(val_dataset)
for k, v in eval_metrics.items():
    if isinstance(v, (int, float)):
        metrics[f"eval_{k}"] = float(v)
    else:
        metrics[f"eval_{k}"] = v

log_history = trainer.state.log_history

log_obj = {
    "training_args": training_args.to_dict(),
    "final_metrics": metrics,
    "log_history": log_history,
}

log_path = os.path.join(OUTPUT_DIR, "training_log_allQA_with_val_3epoch.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log dosyası kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

# ===================== 8. ADAPTÖR + PROCESSOR KAYDET =====================

ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_3epoch")
trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Eğitim tamamlandı. Adaptörler '{ADAPTER_PATH}' klasörüne kaydedildi.")


In [ ]:
# @title
import os
import time
import json
from datetime import timedelta
import sys
import re
import string

import pandas as pd
import torch

from PIL import Image as PImage

import evaluate
from rapidfuzz import fuzz
from sklearn.metrics import precision_recall_fscore_support

from transformers import LlavaForConditionalGeneration, LlavaProcessor, BitsAndBytesConfig
from peft import PeftModel

# ===================== TEMEL BİLGİ / GPU =====================

print("Python:", sys.version)
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yap.")
    sys.exit(0)

device = torch.device("cuda")
print("Kullanılan cihaz:", device)

# ===================== PATHLER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TEST_QA_FILE = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Images")
IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_3epoch")
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_3epoch")

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

# ===================== MODEL + PROCESSOR YÜKLE =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("\n🔹 Base model yükleniyor...")
base_model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

print("🔹 LoRA adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()
model.to(device)

print("🔹 Processor yükleniyor...")
processor = LlavaProcessor.from_pretrained(OUTPUT_DIR)

# ===================== TEST DOSYASINI YÜKLE =====================

print("\n🔹 TEST TXT okunuyor:", TEST_QA_FILE)

df_test = pd.read_csv(
    TEST_QA_FILE,
    sep="|",
    header=None,
    names=["image_id", "category", "question", "answer"],
    encoding="latin-1",
)

df_test["image_id"] = df_test["image_id"].str.strip()
df_test["question"] = df_test["question"].str.strip()
df_test["answer"]   = df_test["answer"].str.strip()

df_test["image_path"] = df_test["image_id"].apply(
    lambda x: os.path.join(TEST_IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

df_test = df_test[df_test["image_path"].apply(os.path.exists)].reset_index(drop=True)

if len(df_test) == 0:
    raise RuntimeError("Hiç geçerli test örneği bulunamadı, TEST_IMAGE_DIR veya uzantı yanlış olabilir.")

print(f"Toplam test örneği : {len(df_test)}")
print(f"Eşsiz test görseli: {df_test['image_id'].nunique()}")

# ===================== NORMALİZE FONKSİYONU =====================

def normalize_answer(s: str) -> str:
    """SQuAD tarzı normalize: lower, noktalama, artikeller, ekstra boşluklar."""
    def lower(text):
        return text.lower()

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    return white_space_fix(remove_articles(remove_punc(lower(s))))

# ===================== INFERENCE + METRİKLER =====================

all_results = []
correct_strict = 0
normalized_matches = 0

pred_texts = []
gt_texts   = []

print("\n📄 Tüm test seti üzerinde inference başlıyor...")

start_inf = time.time()

for idx, row in df_test.iterrows():
    image_id    = row["image_id"]
    question    = row["question"]
    true_answer = row["answer"]
    image_path  = row["image_path"]

    img = PImage.open(image_path).convert("RGB")

    prompt_text = f"USER: <image>\n{question}\nASSISTANT:"

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )[0].strip()

    pred_texts.append(pred)
    gt_texts.append(true_answer)

    is_match_strict = (pred.lower().strip() == true_answer.lower().strip())
    if is_match_strict:
        correct_strict += 1

    if normalize_answer(pred) == normalize_answer(true_answer):
        normalized_matches += 1

    all_results.append(
        {
            "index": int(idx) + 1,
            "image_id": image_id,
            "image_path": image_path,
            "category": row["category"],
            "question": question,
            "ground_truth": true_answer,
            "predicted": pred,
            "match_strict": bool(is_match_strict),
        }
    )

    if (idx + 1) % 50 == 0:
        print(f"  > {idx+1}/{len(df_test)} örnek tamamlandı.")

end_inf = time.time()
total_inf_sec = end_inf - start_inf

n = len(df_test)
accuracy_strict     = (correct_strict / n) * 100 if n > 0 else 0.0
accuracy_normalized = (normalized_matches / n) * 100 if n > 0 else 0.0

# ---------- Fuzzy metrikler ----------

fuzzy_scores_partial = [
    fuzz.partial_ratio(p.lower(), g.lower()) for p, g in zip(pred_texts, gt_texts)
]
fuzzy_acc_80    = sum(s >= 80 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
fuzzy_acc_90    = sum(s >= 90 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
avg_fuzzy_score = sum(fuzzy_scores_partial) / n if n > 0 else 0.0

print("\n📏 Global metrikler hesaplanıyor (BLEU, BLEU-1, ROUGE, METEOR, BERTScore, P/R/F1)...")

bleu_metric      = evaluate.load("bleu")
rouge_metric     = evaluate.load("rouge")
meteor_metric    = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

# BLEU (default: BLEU-4) + BLEU-1 (1-gram precision)
bleu_res = bleu_metric.compute(
    predictions=pred_texts,
    references=[[g] for g in gt_texts],
)
bleu_score = bleu_res["bleu"]          # klasik BLEU (0-1 arası)
bleu_p1    = bleu_res["precisions"][0] # yüzde cinsinden 1-gram precision
bleu_1     = bleu_p1 / 100.0           # 0-1 arası BLEU-1

rouge_res = rouge_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
)
meteor_res = meteor_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
)
bertscore_res = bertscore_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
    lang="en",
)

avg_bertscore_f1 = float(sum(bertscore_res["f1"]) / len(bertscore_res["f1"])) if n > 0 else 0.0

# ---------- Ek: sınıflandırma tarzı Precision / Recall / F1 (macro) ----------

gt_norm  = [normalize_answer(t) for t in gt_texts]
pred_norm = [normalize_answer(t) for t in pred_texts]

prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
    gt_norm,
    pred_norm,
    average="macro",
    zero_division=0,
)

# ===================== METRİK ÖZETİ =====================

print("\n==== METRİK ÖZETİ (AllQA + Val ile eğitilmiş model, Official Test) ====")
print(f"Strict Exact Match Accuracy       : {accuracy_strict:.2f}%")
print(f"Normalized Exact Match Accuracy   : {accuracy_normalized:.2f}%")
print(f"Fuzzy Partial Acc (>=80)          : {fuzzy_acc_80:.2f}%")
print(f"Fuzzy Partial Acc (>=90)          : {fuzzy_acc_90:.2f}%")
print(f"Avg Fuzzy Partial Score           : {avg_fuzzy_score:.2f}")
print(f"BLEU (4-gram)                     : {bleu_score:.4f}")
print(f"BLEU-1 (1-gram precision)         : {bleu_1:.4f}")
print(f"ROUGE-L                           : {rouge_res['rougeL']:.4f}")
print(f"METEOR                            : {meteor_res['meteor']:.4f}")
print(f"BERTScore Avg F1                  : {avg_bertscore_f1:.4f}")
print(f"Macro Precision (normalized EM)   : {prec_macro:.4f}")
print(f"Macro Recall (normalized EM)      : {rec_macro:.4f}")
print(f"Macro F1 (normalized EM)          : {f1_macro:.4f}")
print(f"Toplam inference süresi (sn)      : {total_inf_sec:.2f}")

# ===================== JSON LOG =====================

test_output = {
    "total_test_samples": int(n),
    "metrics": {
        "exact_match_strict_percent": float(accuracy_strict),
        "exact_match_normalized_percent": float(accuracy_normalized),
        "fuzzy_partial_acc_80_percent": float(fuzzy_acc_80),
        "fuzzy_partial_acc_90_percent": float(fuzzy_acc_90),
        "fuzzy_partial_avg_score": float(avg_fuzzy_score),

        "bleu": float(bleu_score),
        "bleu_1": float(bleu_1),
        "rougeL": float(rouge_res["rougeL"]),
        "meteor": float(meteor_res["meteor"]),
        "bertscore_avg_f1": float(avg_bertscore_f1),

        "precision_macro": float(prec_macro),
        "recall_macro": float(rec_macro),
        "f1_macro": float(f1_macro),
    },
    "inference_time_sec": float(total_inf_sec),
    "inference_time_str": str(timedelta(seconds=int(total_inf_sec))),
    "samples": all_results,
}

test_log_path = os.path.join(OUTPUT_DIR, "test_predictions_with_metrics_VQAMed2019_Test.json")
with open(test_log_path, "w", encoding="utf-8") as f:
    json.dump(test_output, f, indent=2, ensure_ascii=False)

print(f"\n📁 Test prediction + metrikler kaydedildi: {test_log_path}")
print("✅ Test script bitti.")


In [ ]:
# @title
# GROUPED QA



import os
import time
import json
from datetime import timedelta
import sys
import re
import string

import pandas as pd
import torch

from PIL import Image as PImage

import evaluate
from rapidfuzz import fuzz
from sklearn.metrics import precision_recall_fscore_support

from transformers import LlavaForConditionalGeneration, LlavaProcessor, BitsAndBytesConfig
from peft import PeftModel

# ===================== TEMEL BİLGİ / GPU =====================

print("Python:", sys.version)
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yap.")
    sys.exit(0)

device = torch.device("cuda")
print("Kullanılan cihaz:", device)

# ===================== PATHLER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TEST_QA_FILE = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Images")
IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_3epoch")
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_3epoch")

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = "You are a medical visual question answering assistant. Answer briefly and only based on the image content."

# ===================== MODEL + PROCESSOR YÜKLE =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("\n🔹 Base model yükleniyor...")
base_model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

print("🔹 LoRA adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()
model.to(device)

print("🔹 Processor yükleniyor...")
processor = LlavaProcessor.from_pretrained(OUTPUT_DIR)

# ===================== TEST DOSYASINI YÜKLE =====================

print("\n🔹 TEST TXT okunuyor:", TEST_QA_FILE)

df_test = pd.read_csv(
    TEST_QA_FILE,
    sep="|",
    header=None,
    names=["image_id", "category", "question", "answer"],
    encoding="latin-1",
)

df_test["image_id"] = df_test["image_id"].str.strip()
df_test["question"] = df_test["question"].str.strip()
df_test["answer"]   = df_test["answer"].str.strip()

df_test["image_path"] = df_test["image_id"].apply(
    lambda x: os.path.join(TEST_IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

df_test = df_test[df_test["image_path"].apply(os.path.exists)].reset_index(drop=True)

if len(df_test) == 0:
    raise RuntimeError("Hiç geçerli test örneği bulunamadı, TEST_IMAGE_DIR veya uzantı yanlış olabilir.")

print(f"Toplam test örneği : {len(df_test)}")
print(f"Eşsiz test görseli: {df_test['image_id'].nunique()}")

# ===================== NORMALİZE FONKSİYONU =====================

def normalize_answer(s: str) -> str:
    """SQuAD tarzı normalize: lower, noktalama, artikeller, ekstra boşluklar."""
    def lower(text):
        return text.lower()

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    return white_space_fix(remove_articles(remove_punc(lower(s))))

# ===================== INFERENCE + METRİKLER =====================

all_results = []          # eski flat liste (istersen kullanırsın)
grouped_by_image = {}     # 🔥 yeni: image bazlı grup

correct_strict = 0
normalized_matches = 0

pred_texts = []
gt_texts   = []

print("\n📄 Tüm test seti üzerinde inference başlıyor...")

start_inf = time.time()

for idx, row in df_test.iterrows():
    image_id    = row["image_id"]
    question    = row["question"]
    true_answer = row["answer"]
    image_path  = row["image_path"]

    img = PImage.open(image_path).convert("RGB")

    prompt_text = f"USER: <image>\n{question}\nASSISTANT:"

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )[0].strip()

    pred_texts.append(pred)
    gt_texts.append(true_answer)

    # strict match
    is_match_strict = (pred.lower().strip() == true_answer.lower().strip())
    if is_match_strict:
        correct_strict += 1

    # normalized match
    norm_pred = normalize_answer(pred)
    norm_true = normalize_answer(true_answer)
    is_match_norm = (norm_pred == norm_true)
    if is_match_norm:
        normalized_matches += 1

    # per-sample fuzzy score (global metrik yine eski şekilde hesaplanacak)
    fuzzy_score = fuzz.partial_ratio(pred.lower(), true_answer.lower())

    # 🔹 Eski flat sonuç (dokunmadım, sadece 2 field ekledim)
    all_results.append(
        {
            "index": int(idx) + 1,
            "image_id": image_id,
            "image_path": image_path,
            "category": row["category"],
            "question": question,
            "ground_truth": true_answer,
            "predicted": pred,
            "match_strict": bool(is_match_strict),
            "match_normalized": bool(is_match_norm),
            "fuzzy_partial_score": float(fuzzy_score),
        }
    )

    # 🔥 Yeni: image_id bazlı grouping
    if image_id not in grouped_by_image:
        grouped_by_image[image_id] = {
            "image_id": image_id,
            "image_path": image_path,
            "qas": []
        }

    grouped_by_image[image_id]["qas"].append(
        {
            "index": int(idx) + 1,
            "category": row["category"],
            "question": question,
            "ground_truth": true_answer,
            "predicted": pred,
            "match_strict": bool(is_match_strict),
            "match_normalized": bool(is_match_norm),
            "fuzzy_partial_score": float(fuzzy_score),
        }
    )

    if (idx + 1) % 50 == 0:
        print(f"  > {idx+1}/{len(df_test)} örnek tamamlandı.")

end_inf = time.time()
total_inf_sec = end_inf - start_inf

n = len(df_test)
accuracy_strict     = (correct_strict / n) * 100 if n > 0 else 0.0
accuracy_normalized = (normalized_matches / n) * 100 if n > 0 else 0.0

# ---------- Fuzzy metrikler (ESKİ HALİYLE, DEĞİŞTİRMEDİM) ----------

fuzzy_scores_partial = [
    fuzz.partial_ratio(p.lower(), g.lower()) for p, g in zip(pred_texts, gt_texts)
]
fuzzy_acc_80    = sum(s >= 80 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
fuzzy_acc_90    = sum(s >= 90 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
avg_fuzzy_score = sum(fuzzy_scores_partial) / n if n > 0 else 0.0

print("\n📏 Global metrikler hesaplanıyor (BLEU, BLEU-1, ROUGE, METEOR, BERTScore, P/R/F1)...")

bleu_metric      = evaluate.load("bleu")
rouge_metric     = evaluate.load("rouge")
meteor_metric    = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

# BLEU (default: BLEU-4) + BLEU-1 (1-gram precision)
bleu_res = bleu_metric.compute(
    predictions=pred_texts,
    references=[[g] for g in gt_texts],
)
bleu_score = bleu_res["bleu"]          # klasik BLEU (0-1 arası)
bleu_p1    = bleu_res["precisions"][0] # yüzde cinsinden 1-gram precision
bleu_1     = bleu_p1 / 100.0           # 0-1 arası BLEU-1

rouge_res = rouge_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
)
meteor_res = meteor_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
)
bertscore_res = bertscore_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
    lang="en",
)

avg_bertscore_f1 = float(sum(bertscore_res["f1"]) / len(bertscore_res["f1"])) if n > 0 else 0.0

# ---------- Ek: sınıflandırma tarzı Precision / Recall / F1 (macro) ----------

gt_norm  = [normalize_answer(t) for t in gt_texts]
pred_norm = [normalize_answer(t) for t in pred_texts]

prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
    gt_norm,
    pred_norm,
    average="macro",
    zero_division=0,
)

# ===================== METRİK ÖZETİ =====================

print("\n==== METRİK ÖZETİ (AllQA + Val ile eğitilmiş model, Official Test) ====")
print(f"Strict Exact Match Accuracy       : {accuracy_strict:.2f}%")
print(f"Normalized Exact Match Accuracy   : {accuracy_normalized:.2f}%")
print(f"Fuzzy Partial Acc (>=80)          : {fuzzy_acc_80:.2f}%")
print(f"Fuzzy Partial Acc (>=90)          : {fuzzy_acc_90:.2f}%")
print(f"Avg Fuzzy Partial Score           : {avg_fuzzy_score:.2f}")
print(f"BLEU (4-gram)                     : {bleu_score:.4f}")
print(f"BLEU-1 (1-gram precision)         : {bleu_1:.4f}")
print(f"ROUGE-L                           : {rouge_res['rougeL']:.4f}")
print(f"METEOR                            : {meteor_res['meteor']:.4f}")
print(f"BERTScore Avg F1                  : {avg_bertscore_f1:.4f}")
print(f"Macro Precision (normalized EM)   : {prec_macro:.4f}")
print(f"Macro Recall (normalized EM)      : {rec_macro:.4f}")
print(f"Macro F1 (normalized EM)          : {f1_macro:.4f}")
print(f"Toplam inference süresi (sn)      : {total_inf_sec:.2f}")

# ===================== JSON LOG =====================

samples_grouped_list = list(grouped_by_image.values())

test_output = {
    "total_test_samples": int(n),
    "total_unique_images": int(len(samples_grouped_list)),
    "metrics": {
        "exact_match_strict_percent": float(accuracy_strict),
        "exact_match_normalized_percent": float(accuracy_normalized),
        "fuzzy_partial_acc_80_percent": float(fuzzy_acc_80),
        "fuzzy_partial_acc_90_percent": float(fuzzy_acc_90),
        "fuzzy_partial_avg_score": float(avg_fuzzy_score),

        "bleu": float(bleu_score),
        "bleu_1": float(bleu_1),
        "rougeL": float(rouge_res["rougeL"]),
        "meteor": float(meteor_res["meteor"]),
        "bertscore_avg_f1": float(avg_bertscore_f1),

        "precision_macro": float(prec_macro),
        "recall_macro": float(rec_macro),
        "f1_macro": float(f1_macro),
    },
    "inference_time_sec": float(total_inf_sec),
    "inference_time_str": str(timedelta(seconds=int(total_inf_sec))),
    # 🔥 Artık 'samples' image bazlı gruplu
    "samples": samples_grouped_list,
    # İstersen eski flat formatla da oynamak için:
    "samples_flat": all_results,
}

test_log_path = os.path.join(OUTPUT_DIR, "test_predictions_with_metrics_VQAMed2019_Test_grouped_by_image.json")
with open(test_log_path, "w", encoding="utf-8") as f:
    json.dump(test_output, f, indent=2, ensure_ascii=False)

print(f"\n📁 Test prediction + metrikler (image bazlı gruplanmış) kaydedildi: {test_log_path}")
print("✅ Test script bitti.")


In [ ]:
# @title
import os
import sys

import pandas as pd
import torch
from PIL import Image as PImage
from IPython.display import display

from transformers import LlavaForConditionalGeneration, LlavaProcessor, BitsAndBytesConfig
from peft import PeftModel

print("Python:", sys.version)
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yap.")
    sys.exit(0)

device = torch.device("cuda")
print("Kullanılan cihaz:", device)

# ===================== PATHLER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TEST_QA_FILE = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Images")
IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_3epoch")
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_3epoch")

MODEL_ID = "llava-hf/llava-1.5-7b-hf"

# ===================== MODEL + PROCESSOR YÜKLE =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("\n🔹 Base model yükleniyor...")
base_model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

print("🔹 LoRA adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()
model.to(device)

print("🔹 Processor yükleniyor...")
processor = LlavaProcessor.from_pretrained(OUTPUT_DIR)

# ===================== TEST DOSYASINI YÜKLE =====================

df_test = pd.read_csv(
    TEST_QA_FILE,
    sep="|",
    header=None,
    names=["image_id", "category", "question", "answer"],
    encoding="latin-1",
)

df_test["image_id"] = df_test["image_id"].str.strip()
df_test["question"] = df_test["question"].str.strip()
df_test["answer"]   = df_test["answer"].str.strip()

df_test["image_path"] = df_test["image_id"].apply(
    lambda x: os.path.join(TEST_IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

df_test = df_test[df_test["image_path"].apply(os.path.exists)].reset_index(drop=True)
print(f"Toplam test örneği (görseli bulunan): {len(df_test)}")

# ===================== SUNUM İÇİN ÖRNEKLER =====================

# 1) Rastgele 5 örnek seç
examples = df_test.sample(n=5, random_state=123).reset_index(drop=True)

# Eğer belirli index’ler istiyorsan (örnek):
# example_indices = [0, 10, 42, 100, 200]
# examples = df_test.iloc[example_indices].reset_index(drop=True)

for i, row in examples.iterrows():
    image_id    = row["image_id"]
    question    = row["question"]
    true_answer = row["answer"]
    image_path  = row["image_path"]

    img = PImage.open(image_path).convert("RGB")

    prompt_text = f"USER: <image>\n{question}\nASSISTANT:"

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )[0].strip()

    print("\n" + "=" * 90)
    print(f"[{i+1}] image_id: {image_id}")
    display(img)
    print(f"Question : {question}")
    print(f"GT       : {true_answer}")
    print(f"Pred     : {pred}")


In [ ]:
# @title
# NEW PROMPT
# NEW PROMPT

import os
import time
import json
from datetime import timedelta
import sys

import pandas as pd
import torch

from PIL import Image as PImage
import PIL  # sürüm için

from datasets import Dataset
import datasets as datasets_module  # sürüm için

import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)

import peft
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ================== SÜRÜMLERİ YAZDIR ==================

def safe_version(name, obj):
    ver = getattr(obj, "__version__", None)
    print(f"{name} version:", ver if ver is not None else "bulunamadı")

print("==== Python ve Temel Bilgiler ====")
print("Python:", sys.version)
print()

print("==== Ana Kütüphaneler ====")
safe_version("pandas", pd)
safe_version("torch", torch)
safe_version("PIL", PIL)
safe_version("datasets", datasets_module)
safe_version("transformers", transformers)
safe_version("peft", peft)

try:
    import bitsandbytes as bnb
    safe_version("bitsandbytes", bnb)
except ImportError:
    bnb = None
    print("bitsandbytes: yüklü değil")

print("\n==== GPU Durumu ====")
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU bulunamadı. Colab'ta Runtime > Change runtime type > GPU yapmayı unutma.")
    sys.exit(0)

device = torch.device("cuda")
print(f"Kullanılan cihaz: {device}")

# ===================== 1. PATH ve PARAMETRELER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TRAIN_QA_FILE = os.path.join(LOCAL_DATA_ROOT, "All_QA_Pairs_train.txt")
VAL_QA_FILE   = os.path.join(LOCAL_DATA_ROOT, "All_QA_Pairs_val.txt")

TRAIN_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "Train_images")
VAL_IMAGE_DIR   = os.path.join(LOCAL_DATA_ROOT, "Val_images")  # klasör adın farklıysa değiştir
IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_4epoch_new_prompt")
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Çıktılar bu klasöre kaydedilecek: {OUTPUT_DIR}")

NUM_TRAIN_EPOCHS = 4   # 4–5 epoch burada makul

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = (
    "You are a medical visual question answering assistant. "
    "Answer briefly and only based on the image content. "
)

# ===================== 2. DATAFRAME -> HF DATASET =====================

def format_and_load_dataset(df: pd.DataFrame) -> Dataset:
    """
    Yeni mantık:
    - 'prompt' içinde sadece SYSTEM + USER + Question + 'ASSISTANT:' var.
    - 'answer' ayrı tutuluyor; loss sadece answer token'larında.
    """
    prompts = []
    for _, row in df.iterrows():
        prompt = (
            f"SYSTEM: {SYSTEM_PROMPT}\n"
            f"USER: <image>\n"
            f"Question: {row['question']}\n"
            f"ASSISTANT:"
        )
        prompts.append(prompt)

    return Dataset.from_dict(
        {
            "image_path": df["image_path"].tolist(),
            "image_id": df["image_id"].tolist(),
            "prompt": prompts,                  # SYSTEM + USER + Question + ASSISTANT:
            "answer": df["answer"].tolist(),    # sadece cevap
            "question": df["question"].tolist(),
            "true_answer": df["answer"].tolist(),
        }
    )

def load_qa_file(path_txt: str, image_dir: str, name: str) -> pd.DataFrame:
    print(f"\n🔹 {name} TXT okunuyor:", path_txt)

    df = pd.read_csv(
        path_txt,
        sep="|",
        header=None,
        names=["image_id", "question", "answer"],
        encoding="latin-1",
    )

    df["image_id"] = df["image_id"].str.strip()
    df["question"] = df["question"].str.strip()
    df["answer"]   = df["answer"].str.strip()

    df["image_path"] = df["image_id"].apply(
        lambda x: os.path.join(image_dir, f"{x}{IMAGE_EXTENSION}")
    )

    df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

    if len(df) == 0:
        raise RuntimeError(f"{name} için hiç geçerli örnek bulunamadı, {image_dir} veya uzantı yanlış olabilir.")

    print(f"{name} QA sayısı        : {len(df)}")
    print(f"{name} eşsiz image_id    : {df['image_id'].nunique()}")
    return df

df_train = load_qa_file(TRAIN_QA_FILE, TRAIN_IMAGE_DIR, "TRAIN")
df_val   = load_qa_file(VAL_QA_FILE,   VAL_IMAGE_DIR,   "VAL")

train_dataset = format_and_load_dataset(df_train)
val_dataset   = format_and_load_dataset(df_val)

print(f"\nEğitim Seti Boyutu : {len(train_dataset)}")
print(f"Validation Seti    : {len(val_dataset)}")

# ===================== 3. MODEL + QLoRA KURULUMU =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

for param in model.vision_tower.parameters():
    param.requires_grad = False

model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

processor = LlavaProcessor.from_pretrained(MODEL_ID)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nEğitilebilir Parametre Sayısı:")
model.print_trainable_parameters()

# ===================== 4. DATA COLLATOR (NEW PROMPT + LABEL MASKING) =====================

def vqa_data_collator(examples):
    """
    - 'prompt' + 'answer' birleştirilip modele veriliyor.
    - Loss sadece answer kısmında: prompt token'ları -100 ile maskeleniyor.
    """
    images   = [PImage.open(e["image_path"]).convert("RGB") for e in examples]
    prompts  = [e["prompt"] for e in examples]
    answers  = [e["answer"] for e in examples]

    full_texts = [f"{p} {a}" for p, a in zip(prompts, answers)]

    proc_out = processor(
        text=full_texts,
        images=images,
        return_tensors="pt",
        padding="longest",
    )

    input_ids      = proc_out["input_ids"]
    attention_mask = proc_out["attention_mask"]
    pixel_values   = proc_out["pixel_values"]

    labels = input_ids.clone()

    tokenizer = processor.tokenizer

    for i, prompt in enumerate(prompts):
        # Prompt uzunluğunu (token) bul
        prompt_ids = tokenizer(prompt, add_special_tokens=True)["input_ids"]
        prompt_len = len(prompt_ids)
        # Prompt kısmını loss dışına al
        labels[i, :prompt_len] = -100

    if tokenizer.pad_token_id is not None:
        labels[labels == tokenizer.pad_token_id] = -100

    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pixel_values": pixel_values,
        "labels": labels,
    }
    return batch

# ===================== 5. TRAININGARGUMENTS + TRAINER =====================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=1,

    eval_strategy="epoch",
    save_strategy="epoch",

    logging_steps=50,
    learning_rate=2e-4,     # istersen sonra 1e-4 + warmup + cosine deneyebiliriz
    optim="adamw_torch",
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=vqa_data_collator,
)

print(f"\n✨ Fine-tuning başlıyor (All train QA, ayrı VAL set, {NUM_TRAIN_EPOCHS} epoch, NEW PROMPT)...")

# ===================== 6. EĞİTİM + LOG DOSYASI =====================

start_time = time.time()
train_result = trainer.train()
end_time = time.time()
total_time_sec = end_time - start_time

metrics = dict(train_result.metrics)
metrics["total_training_time_sec"] = float(total_time_sec)
metrics["total_training_time_str"] = str(timedelta(seconds=int(total_time_sec)))
metrics["num_train_samples"] = len(train_dataset)
metrics["num_eval_samples"] = len(val_dataset)
metrics["num_train_steps"] = train_result.global_step

eval_metrics = trainer.evaluate(val_dataset)
for k, v in eval_metrics.items():
    if isinstance(v, (int, float)):
        metrics[f"eval_{k}"] = float(v)
    else:
        metrics[f"eval_{k}"] = v

log_history = trainer.state.log_history

log_obj = {
    "training_args": training_args.to_dict(),
    "final_metrics": metrics,
    "log_history": log_history,
}

log_path = os.path.join(OUTPUT_DIR, "training_log_allQA_with_val_4epoch_new_prompt.json")
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_obj, f, indent=2)

print(f"\n📄 Log dosyası kaydedildi: {log_path}")
print(f"⏱ Toplam eğitim süresi: {metrics['total_training_time_str']}")

ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_4epoch_new_prompt")
trainer.model.save_pretrained(ADAPTER_PATH)
processor.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Eğitim tamamlandı. Adaptörler '{ADAPTER_PATH}' klasörüne kaydedildi.")



In [ ]:
# @title
# NEW PROMPT
# NEW PROMPT

import os
import time
import json
from datetime import timedelta
import sys
import re
import string

import pandas as pd
import torch

from PIL import Image as PImage

import evaluate
from rapidfuzz import fuzz
from sklearn.metrics import precision_recall_fscore_support

from transformers import LlavaForConditionalGeneration, LlavaProcessor, BitsAndBytesConfig
from peft import PeftModel

print("Python:", sys.version)
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU yok. Colab'ta Runtime > Change runtime type > GPU yap.")
    sys.exit(0)

device = torch.device("cuda")
print("Kullanılan cihaz:", device)

# ===================== PATHLER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TEST_QA_FILE   = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Images")
IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR   = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_4epoch_new_prompt")
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_4epoch_new_prompt")

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = (
    "You are a medical visual question answering assistant. "
    "Answer briefly and only based on the image content. "
    "If possible, respond with a single disease name or a very short phrase."
)

# ===================== MODEL + PROCESSOR YÜKLE =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("\n🔹 Base model yükleniyor...")
base_model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

print("🔹 LoRA adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()
model.to(device)

print("🔹 Processor yükleniyor...")
processor = LlavaProcessor.from_pretrained(OUTPUT_DIR)

# ===================== TEST DOSYASINI YÜKLE =====================

print("\n🔹 TEST TXT okunuyor:", TEST_QA_FILE)

df_test = pd.read_csv(
    TEST_QA_FILE,
    sep="|",
    header=None,
    names=["image_id", "category", "question", "answer"],
    encoding="latin-1",
)

df_test["image_id"] = df_test["image_id"].str.strip()
df_test["question"] = df_test["question"].str.strip()
df_test["answer"]   = df_test["answer"].str.strip()

df_test["image_path"] = df_test["image_id"].apply(
    lambda x: os.path.join(TEST_IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

df_test = df_test[df_test["image_path"].apply(os.path.exists)].reset_index(drop=True)

if len(df_test) == 0:
    raise RuntimeError("Hiç geçerli test örneği bulunamadı, TEST_IMAGE_DIR veya uzantı yanlış olabilir.")

print(f"Toplam test örneği : {len(df_test)}")
print(f"Eşsiz test görseli: {df_test['image_id'].nunique()}")

# ===================== NORMALİZE FONKSİYONU =====================

def normalize_answer(s: str) -> str:
    def lower(text):
        return text.lower()

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    return white_space_fix(remove_articles(remove_punc(lower(s))))

# ===================== INFERENCE + METRİKLER =====================

all_results = []
correct_strict = 0
normalized_matches = 0

pred_texts = []
gt_texts   = []

print("\n📄 Tüm test seti üzerinde inference başlıyor...")

start_inf = time.time()

for idx, row in df_test.iterrows():
    image_id    = row["image_id"]
    question    = row["question"]
    true_answer = row["answer"]
    image_path  = row["image_path"]

    img = PImage.open(image_path).convert("RGB")

    prompt_text = (
        f"SYSTEM: {SYSTEM_PROMPT}\n"
        f"USER: <image>\n"
        f"Question: {question}\n"
        f"ASSISTANT:"
    )

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )[0].strip()

    pred_texts.append(pred)
    gt_texts.append(true_answer)

    is_match_strict = (pred.lower().strip() == true_answer.lower().strip())
    if is_match_strict:
        correct_strict += 1

    if normalize_answer(pred) == normalize_answer(true_answer):
        normalized_matches += 1

    all_results.append(
        {
            "index": int(idx) + 1,
            "image_id": image_id,
            "image_path": image_path,
            "category": row["category"],
            "question": question,
            "ground_truth": true_answer,
            "predicted": pred,
            "match_strict": bool(is_match_strict),
        }
    )

    if (idx + 1) % 50 == 0:
        print(f"  > {idx+1}/{len(df_test)} örnek tamamlandı.")

end_inf = time.time()
total_inf_sec = end_inf - start_inf

n = len(df_test)
accuracy_strict     = (correct_strict / n) * 100 if n > 0 else 0.0
accuracy_normalized = (normalized_matches / n) * 100 if n > 0 else 0.0

# ---------- Fuzzy metrikler ----------

fuzzy_scores_partial = [
    fuzz.partial_ratio(p.lower(), g.lower()) for p, g in zip(pred_texts, gt_texts)
]
fuzzy_acc_80    = sum(s >= 80 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
fuzzy_acc_90    = sum(s >= 90 for s in fuzzy_scores_partial) / n * 100 if n > 0 else 0.0
avg_fuzzy_score = sum(fuzzy_scores_partial) / n if n > 0 else 0.0

print("\n📏 Global metrikler hesaplanıyor (BLEU, BLEU-1, ROUGE, METEOR, BERTScore, P/R/F1)...")

bleu_metric      = evaluate.load("bleu")
rouge_metric     = evaluate.load("rouge")
meteor_metric    = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

bleu_res = bleu_metric.compute(
    predictions=pred_texts,
    references=[[g] for g in gt_texts],
)
bleu_score = bleu_res["bleu"]
bleu_p1    = bleu_res["precisions"][0]
bleu_1     = bleu_p1 / 100.0

rouge_res = rouge_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
)
meteor_res = meteor_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
)
bertscore_res = bertscore_metric.compute(
    predictions=pred_texts,
    references=gt_texts,
    lang="en",
)

avg_bertscore_f1 = float(sum(bertscore_res["f1"]) / len(bertscore_res["f1"])) if n > 0 else 0.0

gt_norm   = [normalize_answer(t) for t in gt_texts]
pred_norm = [normalize_answer(t) for t in pred_texts]

prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
    gt_norm,
    pred_norm,
    average="macro",
    zero_division=0,
)

print("\n==== METRİK ÖZETİ (NEW PROMPT, Official Test) ====")
print(f"Strict Exact Match Accuracy       : {accuracy_strict:.2f}%")
print(f"Normalized Exact Match Accuracy   : {accuracy_normalized:.2f}%")
print(f"Fuzzy Partial Acc (>=80)          : {fuzzy_acc_80:.2f}%")
print(f"Fuzzy Partial Acc (>=90)          : {fuzzy_acc_90:.2f}%")
print(f"Avg Fuzzy Partial Score           : {avg_fuzzy_score:.2f}")
print(f"BLEU (4-gram)                     : {bleu_score:.4f}")
print(f"BLEU-1 (1-gram precision)         : {bleu_1:.4f}")
print(f"ROUGE-L                           : {rouge_res['rougeL']:.4f}")
print(f"METEOR                            : {meteor_res['meteor']:.4f}")
print(f"BERTScore Avg F1                  : {avg_bertscore_f1:.4f}")
print(f"Macro Precision (normalized EM)   : {prec_macro:.4f}")
print(f"Macro Recall (normalized EM)      : {rec_macro:.4f}")
print(f"Macro F1 (normalized EM)          : {f1_macro:.4f}")
print(f"Toplam inference süresi (sn)      : {total_inf_sec:.2f}")

test_output = {
    "total_test_samples": int(n),
    "metrics": {
        "exact_match_strict_percent": float(accuracy_strict),
        "exact_match_normalized_percent": float(accuracy_normalized),
        "fuzzy_partial_acc_80_percent": float(fuzzy_acc_80),
        "fuzzy_partial_acc_90_percent": float(fuzzy_acc_90),
        "fuzzy_partial_avg_score": float(avg_fuzzy_score),

        "bleu": float(bleu_score),
        "bleu_1": float(bleu_1),
        "rougeL": float(rouge_res["rougeL"]),
        "meteor": float(meteor_res["meteor"]),
        "bertscore_avg_f1": float(avg_bertscore_f1),

        "precision_macro": float(prec_macro),
        "recall_macro": float(rec_macro),
        "f1_macro": float(f1_macro),
    },
    "inference_time_sec": float(total_inf_sec),
    "inference_time_str": str(timedelta(seconds=int(total_inf_sec))),
    "samples": all_results,
}

test_log_path = os.path.join(OUTPUT_DIR, "test_predictions_with_metrics_VQAMed2019_Test_new_prompt.json")
with open(test_log_path, "w", encoding="utf-8") as f:
    json.dump(test_output, f, indent=2, ensure_ascii=False)

print(f"\n📁 Test prediction + metrikler kaydedildi: {test_log_path}")
print("✅ Test script bitti (NEW PROMPT).")



In [ ]:
# @title
# NEW PROMPT


# NEW PROMPT

import os
import sys

import pandas as pd
import torch
from PIL import Image as PImage
from IPython.display import display

from transformers import LlavaForConditionalGeneration, LlavaProcessor, BitsAndBytesConfig
from peft import PeftModel

print("Python:", sys.version)
print("torch.cuda.is_available():", torch.cuda.is_available())
if not torch.cuda.is_available():
    print("⚠ GPU yok. Colab'ta Runtime > Change runtime type > GPU yap.")
    sys.exit(0)

device = torch.device("cuda")
print("Kullanılan cihaz:", device)

# ===================== PATHLER =====================

LOCAL_DATA_ROOT = "/content/drive/MyDrive/ImageClef-2019-VQA-Med-Training"

TEST_QA_FILE   = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Questions_w_Ref_Answers.txt")
TEST_IMAGE_DIR = os.path.join(LOCAL_DATA_ROOT, "VQAMed2019_Test_Images")
IMAGE_EXTENSION = ".jpg"

OUTPUT_DIR   = os.path.join(LOCAL_DATA_ROOT, "llava_fine_tune_output_allQA_4epoch_new_prompt")
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_allQA_with_val_4epoch_new_prompt")

MODEL_ID = "llava-hf/llava-1.5-7b-hf"
SYSTEM_PROMPT = (
    "You are a medical visual question answering assistant. "
    "Answer briefly and only based on the image content. "
    "If possible, respond with a single disease name or a very short phrase."
)

# ===================== MODEL + PROCESSOR YÜKLE =====================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("\n🔹 Base model yükleniyor...")
base_model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

print("🔹 LoRA adapter yükleniyor...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()
model.to(device)

print("🔹 Processor yükleniyor...")
processor = LlavaProcessor.from_pretrained(OUTPUT_DIR)

# ===================== TEST DOSYASINI YÜKLE =====================

df_test = pd.read_csv(
    TEST_QA_FILE,
    sep="|",
    header=None,
    names=["image_id", "category", "question", "answer"],
    encoding="latin-1",
)

df_test["image_id"] = df_test["image_id"].str.strip()
df_test["question"] = df_test["question"].str.strip()
df_test["answer"]   = df_test["answer"].str.strip()

df_test["image_path"] = df_test["image_id"].apply(
    lambda x: os.path.join(TEST_IMAGE_DIR, f"{x}{IMAGE_EXTENSION}")
)

df_test = df_test[df_test["image_path"].apply(os.path.exists)].reset_index(drop=True)
print(f"Toplam test örneği (görseli bulunan): {len(df_test)}")

# ===================== SUNUM İÇİN ÖRNEKLER =====================

# 1) Rastgele 5 örnek
examples = df_test.sample(n=5, random_state=123).reset_index(drop=True)

# Eğer belirli index'ler istiyorsan:
# example_indices = [0, 10, 42, 100, 200]
# examples = df_test.iloc[example_indices].reset_index(drop=True)

for i, row in examples.iterrows():
    image_id    = row["image_id"]
    question    = row["question"]
    true_answer = row["answer"]
    image_path  = row["image_path"]

    img = PImage.open(image_path).convert("RGB")

    prompt_text = (
        f"SYSTEM: {SYSTEM_PROMPT}\n"
        f"USER: <image>\n"
        f"Question: {question}\n"
        f"ASSISTANT:"
    )

    inputs = processor(text=prompt_text, images=img, return_tensors="pt").to(device)

    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
        )

    pred = processor.batch_decode(
        generate_ids[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )[0].strip()

    print("\n" + "=" * 90)
    print(f"[{i+1}] image_id: {image_id}")
    display(img)
    print(f"Question : {question}")
    print(f"GT       : {true_answer}")
    print(f"Pred     : {pred}")


In [ ]:
!pip uninstall -y pillow
!pip install pillow==10.4.0


Found existing installation: pillow 10.4.0
Uninstalling pillow-10.4.0:
  Successfully uninstalled pillow-10.4.0
  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
